# This notebooks is for training the vehicles in isaac with waymo scenes

## Getting Waymo Scenes
The first step is building the world, the worlds in isaacsim are built based on the configuration in 

In [ ]:
# chocolate_waymo_builder_with_goal_ring_polling.py
# - 100 worlds demo friendly: no trigger-report subscription (version-stable)
# - Goal: green ring visual + trigger collider (no rigid body)
# - Detection: cheap polling per frame (distance check), remove correct car + ring

from __future__ import annotations

import json
import math
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional, Sequence, Tuple, Union

import numpy as np
from pxr import Gf, Sdf, Usd, UsdGeom, UsdPhysics, Vt

import omni.usd
import omni.kit.app

# ---- PhysX vehicle wizard imports (THE WORKING ONES you pasted) ----
from omni.physxvehicle.scripts.wizards import physxVehicleWizard as VehicleWizard
from omni.physxvehicle.scripts.helpers.UnitScale import UnitScale
from omni.physxvehicle.scripts.commands import PhysXVehicleWizardCreateCommand

# PhysX schema optional (for trigger API if available)
try:
    from pxr import PhysxSchema
except Exception:
    PhysxSchema = None  # type: ignore

ROOT_PATH = "/World"
SHARED_ROOT = ROOT_PATH + "/VehicleShared"


# ----------------------------
# Utilities
# ----------------------------

def _ensure_world_default_prim(stage: Usd.Stage) -> None:
    world = stage.GetPrimAtPath(ROOT_PATH)
    if not world.IsValid():
        world = UsdGeom.Xform.Define(stage, ROOT_PATH).GetPrim()
    if not stage.GetDefaultPrim().IsValid():
        stage.SetDefaultPrim(world)

def _get_unit_scale(stage: Usd.Stage) -> Tuple[UnitScale, float]:
    """Return (UnitScale, meters_per_unit)."""
    meters_per_unit = UsdGeom.GetStageMetersPerUnit(stage)
    if meters_per_unit == 0:
        meters_per_unit = 0.01  # Isaac default often cm
    length_scale = 1.0 / meters_per_unit

    kilograms_per_unit = UsdPhysics.GetStageKilogramsPerUnit(stage)
    if kilograms_per_unit == 0:
        kilograms_per_unit = 1.0
    mass_scale = 1.0 / kilograms_per_unit
    return UnitScale(length_scale, mass_scale), meters_per_unit

def _meters_per_unit(stage: Usd.Stage) -> float:
    mpu = UsdGeom.GetStageMetersPerUnit(stage)
    return float(mpu) if mpu and float(mpu) > 0 else 0.01

def _safe_int(x, default=-1) -> int:
    try:
        return int(x)
    except Exception:
        try:
            return int(float(x))
        except Exception:
            return default

def _safe_float(x, default=None) -> Optional[float]:
    try:
        return float(x)
    except Exception:
        return default

def _quath_from_yaw_z(yaw_rad: float) -> Gf.Quath:
    """Half-precision quat rotation about +Z."""
    half = 0.5 * float(yaw_rad)
    w = float(np.cos(half))
    z = float(np.sin(half))
    try:
        return Gf.Quath(Gf.Half(w), Gf.Half(0.0), Gf.Half(0.0), Gf.Half(z))
    except Exception:
        return Gf.Quath(w, 0.0, 0.0, z)

def _reduce_polyline_xy(points_xyz: np.ndarray, area_thresh: float) -> np.ndarray:
    """Iteratively remove points with small local triangle area (XY) under threshold."""
    pts = np.asarray(points_xyz, dtype=np.float32)
    if pts.shape[0] < 3 or area_thresh <= 0:
        return pts

    keep = np.ones((pts.shape[0],), dtype=bool)
    changed = True

    def tri_area_xy(a, b, c) -> float:
        abx = float(b[0] - a[0]); aby = float(b[1] - a[1])
        acx = float(c[0] - a[0]); acy = float(c[1] - a[1])
        cross = abx * acy - aby * acx
        return 0.5 * abs(cross)

    while changed:
        changed = False
        idxs = np.nonzero(keep)[0]
        if idxs.size < 3:
            break
        for k in range(1, idxs.size - 1):
            i0 = idxs[k - 1]; i1 = idxs[k]; i2 = idxs[k + 1]
            if tri_area_xy(pts[i0], pts[i1], pts[i2]) < area_thresh:
                keep[i1] = False
                changed = True
    return pts[keep]

def _get_world_translation_m(stage: Usd.Stage, prim_path: str) -> Optional[Tuple[float, float, float]]:
    prim = stage.GetPrimAtPath(prim_path)
    if not prim.IsValid():
        return None
    xf = UsdGeom.Xformable(prim)
    if not xf:
        return None
    m = xf.ComputeLocalToWorldTransform(Usd.TimeCode.Default())
    t = m.ExtractTranslation()  # stage units
    mpu = _meters_per_unit(stage)
    return (float(t[0] * mpu), float(t[1] * mpu), float(t[2] * mpu))


# ----------------------------
# Goal manager (polling, no PhysX trigger subscription)
# ----------------------------

class _GoalPollingManager:
    """
    Very stable across Isaac versions:
    - subscribe to app update stream
    - each frame: check distance(car, goal_center) <= radius
    - if yes: remove both car and goal prims
    """
    def __init__(self, stage: Usd.Stage):
        self.stage = stage
        self._sub = None
        # key: goal_root_path
        # val: dict(center=(x,y,z) meters, radius, car_path, agent_id)
        self.goals: Dict[str, Dict[str, Any]] = {}

    def ensure_subscription(self) -> None:
        if self._sub is not None:
            return
        stream = omni.kit.app.get_app().get_update_event_stream()
        self._sub = stream.create_subscription_to_pop(self._on_update, name="goal_polling")

    def add_goal(self, goal_root_path: str, *, center_m: Tuple[float, float, float], radius_m: float,
                 car_root_path: str, agent_id: int) -> None:
        self.goals[goal_root_path] = {
            "center_m": tuple(map(float, center_m)),
            "radius_m": float(radius_m),
            "car_root_path": str(car_root_path),
            "agent_id": int(agent_id),
        }
        self.ensure_subscription()

    def _on_update(self, e) -> None:
        if not self.goals:
            return

        stage = self.stage
        to_remove: List[str] = []

        # iterate over snapshot so we can delete
        for goal_path, info in list(self.goals.items()):
            car_path = info["car_root_path"]
            center = info["center_m"]
            r = float(info["radius_m"])
            r2 = r * r

            car_t = _get_world_translation_m(stage, car_path)
            if car_t is None:
                # car gone -> cleanup goal too
                to_remove.append(goal_path)
                continue

            dx = float(car_t[0] - center[0])
            dy = float(car_t[1] - center[1])
            dz = float(car_t[2] - center[2])
            d2 = dx * dx + dy * dy + dz * dz

            if d2 <= r2:
                # correct car reached its own goal
                try:
                    stage.RemovePrim(Sdf.Path(car_path))
                except Exception:
                    pass
                try:
                    stage.RemovePrim(Sdf.Path(goal_path))
                except Exception:
                    pass
                to_remove.append(goal_path)

        for gp in to_remove:
            self.goals.pop(gp, None)


# module-level singleton per stage (good enough for your demo)
_GOAL_MGR: Optional[_GoalPollingManager] = None
def _goal_mgr(stage: Usd.Stage) -> _GoalPollingManager:
    global _GOAL_MGR
    if _GOAL_MGR is None or _GOAL_MGR.stage != stage:
        _GOAL_MGR = _GoalPollingManager(stage)
    return _GOAL_MGR


# ----------------------------
# Mini-world builder (ONE world)
# ----------------------------

@dataclass
class LocalBounds:
    width_m: float = 200.0
    length_m: float = 200.0
    origin_xy: Tuple[float, float] = (0.0, 0.0)   # local-space center of bounds


class WaymoJsonMiniWorldBuilder:
    """
    Builds ONE mini-world under world_root.

    Behavior:
    - start_in_goal agents become PARKED CARS:
        * static collider on chassis
        * visual wheels only (no wheel colliders)
        * NO goal spawned
    - other agents remain full PhysX vehicles (wizard) + green ring goal
      goal completion is detected via cheap per-frame distance check (no trigger-report API).
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        world_root: str = "/World/MiniWorlds/world_000",
        bounds: Optional[LocalBounds] = None,
        origin_mode: str = "center",  # "center" or "zero"
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.world_root = world_root
        self.bounds = bounds or LocalBounds()
        self.origin_mode = origin_mode

        self._scene_center = np.zeros((3,), dtype=np.float32)

        # ensure root prim exists + subfolders
        UsdGeom.Xform.Define(self.stage, self.world_root)
        self.road_root = f"{self.world_root}/Road"
        self.agents_root = f"{self.world_root}/Agents"
        self.goals_root = f"{self.world_root}/Goals"
        UsdGeom.Xform.Define(self.stage, self.road_root)
        UsdGeom.Xform.Define(self.stage, self.agents_root)
        UsdGeom.Xform.Define(self.stage, self.goals_root)

    # -------- bounds gating in LOCAL coords --------
    def _in_bounds_xy(self, x: float, y: float) -> bool:
        ox, oy = self.bounds.origin_xy
        hw = 0.5 * float(self.bounds.width_m)
        hl = 0.5 * float(self.bounds.length_m)
        return (ox - hw <= x <= ox + hw) and (oy - hl <= y <= oy + hl)

    def _filter_polyline_points(self, pts: np.ndarray) -> np.ndarray:
        if pts.shape[0] == 0:
            return pts
        m = np.array([self._in_bounds_xy(float(p[0]), float(p[1])) for p in pts], dtype=bool)
        return pts[m]

    # -------- origin shift --------
    def _compute_scene_center_from_road(self, polylines: List[Dict[str, Any]]) -> np.ndarray:
        all_pts = []
        for pl in polylines:
            xyz = pl.get("xyz", None)
            if not xyz:
                continue
            arr = np.asarray(xyz, dtype=np.float32)
            if arr.ndim == 2 and arr.shape[1] >= 2 and arr.shape[0] >= 1:
                if arr.shape[1] == 2:
                    arr = np.concatenate([arr, np.zeros((arr.shape[0], 1), dtype=np.float32)], axis=1)
                all_pts.append(arr[:, :3])
        if not all_pts:
            return np.zeros((3,), dtype=np.float32)
        pts = np.concatenate(all_pts, axis=0)
        return pts.mean(axis=0)

    def _to_local_xyz(self, x: float, y: float, z: float) -> Tuple[float, float, float]:
        return (float(x - self._scene_center[0]),
                float(y - self._scene_center[1]),
                float(z - self._scene_center[2]))

    def _is_start_within_goal(self, sx: float, sy: float, sz: float, ex: float, ey: float, ez: float, radius_m: float) -> bool:
        dx = float(ex - sx)
        dy = float(ey - sy)
        dz = float(ez - sz)
        return (dx * dx + dy * dy + dz * dz) <= float(radius_m * radius_m)

    # -------- road segments grouped by type --------
    def build_road_segments_grouped(
        self,
        cfg: Dict[str, Any],
        polyline_reduction_area: float = 0.0,   # 0 disables reduction
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,
        max_segments_per_type: Optional[int] = None,
    ) -> None:
        road = (cfg.get("road", {}) or {})
        polylines = road.get("polylines", []) or []

        # compute scene center (for origin_mode="center")
        if self.origin_mode == "center":
            self._scene_center = self._compute_scene_center_from_road(polylines)
        else:
            self._scene_center = np.zeros((3,), dtype=np.float32)

        # group polylines by type
        by_type: Dict[int, List[np.ndarray]] = {}
        for pl in polylines:
            t = _safe_int(pl.get("type", -1), -1)
            xyz = pl.get("xyz", None)
            if not xyz:
                continue
            pts = np.asarray(xyz, dtype=np.float32)
            if pts.ndim != 2 or pts.shape[0] < 2:
                continue
            if pts.shape[1] == 2:
                pts = np.concatenate([pts, np.zeros((pts.shape[0], 1), dtype=np.float32)], axis=1)

            pts_local = np.array([self._to_local_xyz(p[0], p[1], p[2]) for p in pts[:, :3]], dtype=np.float32)
            pts_local = self._filter_polyline_points(pts_local)
            if pts_local.shape[0] < 2:
                continue

            if polyline_reduction_area > 0 and pts_local.shape[0] >= int(min_points_for_reduction):
                pts_local = _reduce_polyline_xy(pts_local, float(polyline_reduction_area))
                if pts_local.shape[0] < 2:
                    continue

            by_type.setdefault(t, []).append(pts_local)

        # build one PointInstancer per type
        for t, polys in sorted(by_type.items(), key=lambda kv: kv[0]):
            type_root = f"{self.road_root}/Type_{t:02d}"
            instancer_path = f"{type_root}/Segments"

            UsdGeom.Xform.Define(self.stage, type_root)

            seg_positions_py = []
            seg_orients_py = []
            seg_scales_py = []
            proto_indices_py = []

            seg_count = 0
            for poly in polys:
                for i in range(poly.shape[0] - 1):
                    p0 = poly[i]
                    p1 = poly[i + 1]

                    if not (self._in_bounds_xy(float(p0[0]), float(p0[1])) and self._in_bounds_xy(float(p1[0]), float(p1[1]))):
                        continue

                    dx = float(p1[0] - p0[0])
                    dy = float(p1[1] - p0[1])
                    length = float(math.sqrt(dx * dx + dy * dy))
                    if length < 1e-6:
                        continue
                    if length > float(jump_break_m):
                        continue

                    mid = Gf.Vec3f(
                        float((p0[0] + p1[0]) * 0.5),
                        float((p0[1] + p1[1]) * 0.5),
                        float((p0[2] + p1[2]) * 0.5 + z_lift),
                    )
                    yaw = float(math.atan2(dy, dx))
                    q = _quath_from_yaw_z(yaw)
                    scale = Gf.Vec3f(float(length), float(seg_width), float(seg_height))

                    seg_positions_py.append(mid)
                    seg_orients_py.append(q)
                    seg_scales_py.append(scale)
                    proto_indices_py.append(0)

                    seg_count += 1
                    if max_segments_per_type is not None and seg_count >= int(max_segments_per_type):
                        break
                if max_segments_per_type is not None and seg_count >= int(max_segments_per_type):
                    break

            if seg_count == 0:
                continue

            instancer = UsdGeom.PointInstancer.Define(self.stage, instancer_path)
            proto_path = f"{instancer_path}/CubeProto"
            cube = UsdGeom.Cube.Define(self.stage, proto_path)
            cube.GetSizeAttr().Set(1.0)

            instancer.CreatePrototypesRel().SetTargets([cube.GetPath()])
            instancer.GetPositionsAttr().Set(Vt.Vec3fArray(seg_positions_py))
            instancer.GetOrientationsAttr().Set(Vt.QuathArray(seg_orients_py))
            instancer.GetScalesAttr().Set(Vt.Vec3fArray(seg_scales_py))
            instancer.GetProtoIndicesAttr().Set(Vt.IntArray(proto_indices_py))
            instancer.GetPrim().SetCustomDataByKey("road_type", int(t))

    # -------- vehicles + parked cars --------

    def _spawn_vehicle_wizard_under(self, parent_path: str, position_m: Tuple[float, float, float], yaw_deg: float) -> Optional[str]:
        stage = self.stage
        _ensure_world_default_prim(stage)

        parent_xform = UsdGeom.Xform.Define(stage, parent_path)
        xform_api = UsdGeom.XformCommonAPI(parent_xform)

        unit_scale, meters_per_unit = _get_unit_scale(stage)
        x_m, y_m, z_m = position_m
        pos_units = Gf.Vec3d(x_m / meters_per_unit, y_m / meters_per_unit, z_m / meters_per_unit)

        xform_api.SetTranslate(pos_units)
        xform_api.SetRotate(Gf.Vec3f(0.0, 0.0, float(yaw_deg)), UsdGeom.XformCommonAPI.RotationOrderXYZ)

        vehicle_data = VehicleWizard.VehicleData(
            unit_scale,
            VehicleWizard.VehicleData.AXIS_Z,
            VehicleWizard.VehicleData.AXIS_X,
        )
        vehicle_data.rootVehiclePath = parent_path + "/Vehicle"
        vehicle_data.rootSharedPath = SHARED_ROOT

        ret = PhysXVehicleWizardCreateCommand.execute(vehicle_data)
        success = bool(ret[0]) if isinstance(ret, (tuple, list)) and len(ret) > 0 else False
        if not success:
            payload = ret[1] if isinstance(ret, (tuple, list)) and len(ret) > 1 else None
            messages = payload[0] if isinstance(payload, (tuple, list)) and len(payload) > 0 else None
            print("[VEH WIZ FAIL]", parent_path, "messages:", messages)
            return None

        return vehicle_data.rootVehiclePath

    def _spawn_parked_car_visual(
        self,
        path: str,
        *,
        position_m: Tuple[float, float, float],
        yaw_deg: float,
        chassis_size_m: Tuple[float, float, float] = (4.0, 2.0, 1.0),  # L,W,H
        wheel_radius_m: float = 0.35,
        wheel_thickness_m: float = 0.15,
        wheel_inset_x_m: float = 0.35,
        wheel_inset_y_m: float = 0.25,
        ground_clearance_m: float = 0.25,
    ) -> str:
        stage = self.stage
        mpu = _meters_per_unit(stage)

        L, W, H = chassis_size_m
        clearance = float(max(0.0, ground_clearance_m))

        root = UsdGeom.Xform.Define(stage, path)
        xapi = UsdGeom.XformCommonAPI(root)
        xapi.SetTranslate(Gf.Vec3d(position_m[0] / mpu, position_m[1] / mpu, position_m[2] / mpu))
        xapi.SetRotate(Gf.Vec3f(0.0, 0.0, float(yaw_deg)), UsdGeom.XformCommonAPI.RotationOrderXYZ)

        chassis = UsdGeom.Cube.Define(stage, path + "/Chassis")
        chassis.GetSizeAttr().Set(1.0)

        capi = UsdGeom.XformCommonAPI(chassis)
        capi.SetTranslate(Gf.Vec3d(0.0, 0.0, float((0.5 * H + clearance) / mpu)))
        capi.SetScale(Gf.Vec3f(float(L / mpu), float(W / mpu), float(H / mpu)))

        # cheap static collider
        UsdPhysics.CollisionAPI.Apply(chassis.GetPrim())

        wheels_root = UsdGeom.Xform.Define(stage, path + "/Wheels")

        half_L = 0.5 * L
        half_W = 0.5 * W
        wheel_inset_x_m = float(max(0.0, min(wheel_inset_x_m, half_L - 1e-3)))
        wheel_inset_y_m = float(max(0.0, min(wheel_inset_y_m, half_W - 1e-3)))

        wx_front = +(half_L - wheel_inset_x_m)
        wx_rear  = -(half_L - wheel_inset_x_m)
        wy_left  = +(half_W - wheel_inset_y_m)
        wy_right = -(half_W - wheel_inset_y_m)

        wz = float(wheel_radius_m)  # wheels touch ground
        wheel_rot = Gf.Vec3f(90.0, 0.0, 0.0)

        wheel_specs = {
            "FL": (wx_front, wy_left,  wz),
            "FR": (wx_front, wy_right, wz),
            "RL": (wx_rear,  wy_left,  wz),
            "RR": (wx_rear,  wy_right, wz),
        }

        for name, (wx, wy, wz_m) in wheel_specs.items():
            wpath = wheels_root.GetPath().pathString + f"/{name}"
            wheel = UsdGeom.Cylinder.Define(stage, wpath)
            wheel.CreateRadiusAttr(float(wheel_radius_m / mpu))
            wheel.CreateHeightAttr(float(wheel_thickness_m / mpu))

            wapi = UsdGeom.XformCommonAPI(wheel)
            wapi.SetTranslate(Gf.Vec3d(float(wx / mpu), float(wy / mpu), float(wz_m / mpu)))
            wapi.SetRotate(wheel_rot, UsdGeom.XformCommonAPI.RotationOrderXYZ)

        root_prim = stage.GetPrimAtPath(path)
        root_prim.SetCustomDataByKey("parked_car", True)
        return path

    # -------- goal ring (visual) + trigger collider (no RB) --------

    def _spawn_goal_ring_with_trigger(
        self,
        goal_root_path: str,
        *,
        center_m: Tuple[float, float, float],
        radius_m: float,
        ring_tube_radius_m: float = 0.12,
        trigger_height_m: float = 0.6,
        trigger_radius_scale: float = 1.05,
        color_rgb: Tuple[float, float, float] = (0.1, 0.9, 0.2),
        z_on_ground: Optional[float] = 0.0,   # enforce z
    ) -> str:
        """
        Creates:
          - Visual ring (green) at z=z_on_ground (or center_m[2] if None)
          - Trigger collider cylinder (invisible) that overlaps when car reaches goal
        We DO NOT rely on PhysX trigger report API; collision is optional for debug only.
        """
        stage = self.stage
        mpu = _meters_per_unit(stage)

        cx, cy, cz = map(float, center_m)
        if z_on_ground is not None:
            cz = float(z_on_ground)

        root = UsdGeom.Xform.Define(stage, goal_root_path)
        UsdGeom.XformCommonAPI(root).SetTranslate(Gf.Vec3d(cx / mpu, cy / mpu, cz / mpu))

        # Visual ring:
        # Prefer UsdGeom.Torus if available; otherwise fall back to a thin cylinder "disk".
        ring_path = goal_root_path + "/Ring"
        if hasattr(UsdGeom, "Torus"):
            torus = UsdGeom.Torus.Define(stage, ring_path)
            # Torus attrs are in stage units; keep it simple with scale.
            torus.CreateMajorRadiusAttr(float(radius_m / mpu))
            torus.CreateMinorRadiusAttr(float(ring_tube_radius_m / mpu))
            # Rotate so it lies on ground (torus is usually around Z already, but keep consistent)
            UsdGeom.XformCommonAPI(torus).SetRotate(Gf.Vec3f(0.0, 90.0, 0.0), UsdGeom.XformCommonAPI.RotationOrderXYZ)
            ring_prim = torus.GetPrim()
        else:
            # Fallback visual: very thin cylinder
            cyl = UsdGeom.Cylinder.Define(stage, ring_path)
            cyl.CreateRadiusAttr(float(radius_m / mpu))
            cyl.CreateHeightAttr(float(0.05 / mpu))
            UsdGeom.XformCommonAPI(cyl).SetRotate(Gf.Vec3f(0.0, 0.0, 0.0), UsdGeom.XformCommonAPI.RotationOrderXYZ)
            ring_prim = cyl.GetPrim()

        # green color
        try:
            gprim = UsdGeom.Gprim(ring_prim)
            gprim.CreateDisplayColorAttr([Gf.Vec3f(*map(float, color_rgb))])
        except Exception:
            pass

        # Trigger collider (no rigid body)
        trig_path = goal_root_path + "/Trigger"
        trig = UsdGeom.Cylinder.Define(stage, trig_path)
        trig.CreateRadiusAttr(float((radius_m * trigger_radius_scale) / mpu))
        trig.CreateHeightAttr(float(trigger_height_m / mpu))
        # align vertical (default cylinder axis is +Z)
        # Move trigger so it spans above ground:
        UsdGeom.XformCommonAPI(trig).SetTranslate(Gf.Vec3d(0.0, 0.0, float((0.5 * trigger_height_m) / mpu)))

        trig_prim = trig.GetPrim()
        UsdPhysics.CollisionAPI.Apply(trig_prim)
        UsdGeom.Imageable(trig_prim).MakeInvisible()

        try:
            UsdPhysics.CollisionAPI(trig_prim).CreateCollisionEnabledAttr(False)
        except Exception:
            pass

        # Tag
        stage.GetPrimAtPath(goal_root_path).SetCustomDataByKey("is_goal", True)
        stage.GetPrimAtPath(goal_root_path).SetCustomDataByKey("goal_radius_m", float(radius_m))
        stage.GetPrimAtPath(goal_root_path).SetCustomDataByKey("goal_center_m", (float(cx), float(cy), float(cz)))

        return goal_root_path

    # -------- agents + goals --------

    def build_agents_with_goals(
        self,
        cfg: Dict[str, Any],
        max_agents: Optional[int] = None,
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,
        require_goal_in_bounds: bool = True,

        # parked-car knobs:
        parked_if_start_in_goal: bool = True,
        start_goal_thresh_m: Optional[float] = None,  # if None uses goal_radius_m
        parked_ground_z_m: float = 0.0,
        parked_chassis_size_m: Tuple[float, float, float] = (4.0, 2.0, 1.0),
        parked_wheel_radius_m: float = 0.35,
        parked_wheel_thickness_m: float = 0.15,
        parked_wheel_inset_x_m: float = 0.35,
        parked_wheel_inset_y_m: float = 0.25,
        parked_ground_clearance_m: float = 0.25,

        # goal ring knobs:
        goal_ring_z_m: float = 0.0,  # enforce on ground
        goal_ring_tube_radius_m: float = 0.12,
        goal_trigger_height_m: float = 0.6,
    ) -> None:
        agents = (cfg.get("agents", {}) or {}).get("items", []) or []
        kept = 0
        skipped = 0

        mgr = _goal_mgr(self.stage)  # ensure singleton

        for idx, a in enumerate(agents):
            if max_agents is not None and kept >= int(max_agents):
                break

            s = a.get("start", {}) or {}
            e = a.get("end", {}) or {}

            sx = _safe_float(s.get("x", None))
            sy = _safe_float(s.get("y", None))
            sz = _safe_float(s.get("z", 0.0), 0.0)
            syaw = _safe_float(s.get("yaw", 0.0), 0.0)  # radians

            ex = _safe_float(e.get("x", None))
            ey = _safe_float(e.get("y", None))
            ez = _safe_float(e.get("z", 0.0), 0.0)

            if sx is None or sy is None or ex is None or ey is None:
                skipped += 1
                continue

            # LOCAL coords
            sx, sy, sz = self._to_local_xyz(sx, sy, sz)
            ex, ey, ez = self._to_local_xyz(ex, ey, ez)

            # bounds gate
            if not self._in_bounds_xy(float(sx), float(sy)):
                skipped += 1
                continue
            if require_goal_in_bounds and (not self._in_bounds_xy(float(ex), float(ey))):
                skipped += 1
                continue

            thresh = float(start_goal_thresh_m) if start_goal_thresh_m is not None else float(goal_radius_m)
            start_in_goal = self._is_start_within_goal(sx, sy, sz, ex, ey, ez, thresh)

            agent_id = _safe_int(a.get("agent_id", idx), idx)
            agent_path = f"{self.agents_root}/Agent_{kept:04d}_id{agent_id}"
            UsdGeom.Xform.Define(self.stage, agent_path)

            yaw_deg = float(np.degrees(float(syaw)))

            # start_in_goal => parked car (NO GOAL)
            if parked_if_start_in_goal and start_in_goal:
                parked_path = f"{agent_path}/ParkedCar"
                self._spawn_parked_car_visual(
                    parked_path,
                    position_m=(float(sx), float(sy), float(parked_ground_z_m)),
                    yaw_deg=yaw_deg,
                    chassis_size_m=parked_chassis_size_m,
                    wheel_radius_m=parked_wheel_radius_m,
                    wheel_thickness_m=parked_wheel_thickness_m,
                    wheel_inset_x_m=parked_wheel_inset_x_m,
                    wheel_inset_y_m=parked_wheel_inset_y_m,
                    ground_clearance_m=parked_ground_clearance_m,
                )
                prim_outer = self.stage.GetPrimAtPath(parked_path)
                prim_outer.SetCustomDataByKey("agent_id", int(agent_id))
                prim_outer.SetCustomDataByKey("agent_type", int(_safe_int(a.get("agent_type", -1), -1)))
                prim_outer.SetCustomDataByKey("track_idx", int(_safe_int(a.get("track_idx", kept), kept)))
                prim_outer.SetCustomDataByKey("start_in_goal", True)
                prim_outer.SetCustomDataByKey("controllable", False)

                kept += 1
                continue

            # otherwise: full PhysX vehicle
            veh_parent = f"{agent_path}/Vehicle_Parent"
            veh_outer = self._spawn_vehicle_wizard_under(
                veh_parent,
                position_m=(float(sx), float(sy), float(spawn_z_m)),
                yaw_deg=yaw_deg,
            )
            if veh_outer is None:
                skipped += 1
                continue

            prim_outer = self.stage.GetPrimAtPath(veh_outer)
            prim_outer.SetCustomDataByKey("agent_id", int(agent_id))
            prim_outer.SetCustomDataByKey("agent_type", int(_safe_int(a.get("agent_type", -1), -1)))
            prim_outer.SetCustomDataByKey("track_idx", int(_safe_int(a.get("track_idx", kept), kept)))
            prim_outer.SetCustomDataByKey("start_in_goal", False)
            prim_outer.SetCustomDataByKey("controllable", True)

            # GOAL ring + trigger collider (no RB), enforced z=goal_ring_z_m
            goal_path = f"{self.goals_root}/Goal_{kept:04d}_id{agent_id}"
            self._spawn_goal_ring_with_trigger(
                goal_path,
                center_m=(float(ex), float(ey), float(ez)),
                radius_m=float(goal_radius_m),
                ring_tube_radius_m=float(goal_ring_tube_radius_m),
                trigger_height_m=float(goal_trigger_height_m),
                z_on_ground=float(goal_ring_z_m),
            )

            # register with polling manager (correct car only)
            mgr.add_goal(
                goal_path,
                center_m=(float(ex), float(ey), float(goal_ring_z_m)),
                radius_m=float(goal_radius_m),
                car_root_path=str(veh_outer),
                agent_id=int(agent_id),
            )

            kept += 1

        print(f"[MiniWorldBuilder] Agents kept={kept} skipped={skipped}")

    # -------- entry point --------

    def build_from_json(
        self,
        json_path: Union[str, Path],
        *,
        max_agents: int = 50,

        # road params:
        polyline_reduction_area: float = 0.0,
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,

        # agent params:
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,

        # parked car params:
        parked_if_start_in_goal: bool = True,
        start_goal_thresh_m: Optional[float] = None,
        parked_ground_z_m: float = 0.0,
        parked_chassis_size_m: Tuple[float, float, float] = (4.0, 2.0, 1.0),
        parked_wheel_radius_m: float = 0.35,
        parked_wheel_thickness_m: float = 0.15,
        parked_wheel_inset_x_m: float = 0.35,
        parked_wheel_inset_y_m: float = 0.25,
        parked_ground_clearance_m: float = 0.25,

        # goal ring params:
        goal_ring_z_m: float = 0.0,
        goal_ring_tube_radius_m: float = 0.12,
        goal_trigger_height_m: float = 0.6,
    ) -> None:
        p = Path(json_path).expanduser().resolve()
        with p.open("r", encoding="utf-8") as f:
            cfg = json.load(f)

        root_prim = self.stage.GetPrimAtPath(self.world_root)
        root_prim.SetCustomDataByKey("scene_json", str(p))
        root_prim.SetCustomDataByKey("origin_mode", str(self.origin_mode))
        root_prim.SetCustomDataByKey("bounds_w_m", float(self.bounds.width_m))
        root_prim.SetCustomDataByKey("bounds_l_m", float(self.bounds.length_m))

        self.build_road_segments_grouped(
            cfg,
            polyline_reduction_area=polyline_reduction_area,
            min_points_for_reduction=min_points_for_reduction,
            jump_break_m=jump_break_m,
            seg_width=seg_width,
            seg_height=seg_height,
            z_lift=z_lift,
        )
        self.build_agents_with_goals(
            cfg,
            max_agents=max_agents,
            spawn_z_m=spawn_z_m,
            goal_radius_m=goal_radius_m,
            require_goal_in_bounds=True,

            parked_if_start_in_goal=parked_if_start_in_goal,
            start_goal_thresh_m=start_goal_thresh_m,
            parked_ground_z_m=parked_ground_z_m,
            parked_chassis_size_m=parked_chassis_size_m,
            parked_wheel_radius_m=parked_wheel_radius_m,
            parked_wheel_thickness_m=parked_wheel_thickness_m,
            parked_wheel_inset_x_m=parked_wheel_inset_x_m,
            parked_wheel_inset_y_m=parked_wheel_inset_y_m,
            parked_ground_clearance_m=parked_ground_clearance_m,

            goal_ring_z_m=goal_ring_z_m,
            goal_ring_tube_radius_m=goal_ring_tube_radius_m,
            goal_trigger_height_m=goal_trigger_height_m,
        )


# ----------------------------
# Chocolate-bar multi-world constructor (GRID of mini-worlds)
# ----------------------------

@dataclass
class GridLayout:
    world_size_m: Tuple[float, float] = (200.0, 200.0)
    padding_m: float = 0.0
    grid_cols: int = 5
    base_z_m: float = 0.0


class ChocolateBarConstructor:
    """
    Places N mini-world roots in a grid and spawns each from a JSON.
    IMPORTANT: Per-world filtering is LOCAL, grid translation is on the root prim only.
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        root_container: str = "/World/MiniWorlds",
        layout: GridLayout = GridLayout(),
        origin_mode: str = "center",
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.root_container = root_container
        self.layout = layout
        self.origin_mode = origin_mode

        UsdGeom.Xform.Define(self.stage, self.root_container)

    def _world_root_path(self, i: int) -> str:
        return f"{self.root_container}/world_{i:03d}"

    def _world_translation(self, i: int) -> Tuple[float, float, float]:
        cols = int(self.layout.grid_cols)
        sx, sy = self.layout.world_size_m
        pitch_x = float(sx + self.layout.padding_m)
        pitch_y = float(sy + self.layout.padding_m)
        r = i // cols
        c = i % cols
        return (float(c * pitch_x), float(r * pitch_y), float(self.layout.base_z_m))

    def _set_root_translation(self, prim_path: str, txyz: Tuple[float, float, float]) -> None:
        prim = self.stage.GetPrimAtPath(prim_path)
        if not prim.IsValid():
            UsdGeom.Xform.Define(self.stage, prim_path)
            prim = self.stage.GetPrimAtPath(prim_path)
        xform = UsdGeom.XformCommonAPI(prim)
        xform.SetTranslate(Gf.Vec3d(*txyz))

    def clear_all(self) -> None:
        self.stage.RemovePrim(Sdf.Path(self.root_container))
        UsdGeom.Xform.Define(self.stage, self.root_container)

    def build(
        self,
        json_paths: Sequence[Union[str, Path]],
        world_count: int,
        *,
        bounds_size_m: float = 200.0,
        max_agents_per_world: int = 50,

        # road params:
        polyline_reduction_area: float = 0.0,
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,

        # agent params:
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,

        # parked car params:
        parked_if_start_in_goal: bool = True,
        start_goal_thresh_m: Optional[float] = None,
        parked_ground_z_m: float = 0.0,
        parked_chassis_size_m: Tuple[float, float, float] = (4.0, 2.0, 1.0),
        parked_wheel_radius_m: float = 0.35,
        parked_wheel_thickness_m: float = 0.15,
        parked_wheel_inset_x_m: float = 0.35,
        parked_wheel_inset_y_m: float = 0.25,
        parked_ground_clearance_m: float = 0.25,

        # goal ring params:
        goal_ring_z_m: float = 0.0,
        goal_ring_tube_radius_m: float = 0.12,
        goal_trigger_height_m: float = 0.6,
    ) -> None:
        json_list = [str(Path(p).expanduser().resolve()) for p in json_paths]
        if len(json_list) == 0:
            raise ValueError("json_paths is empty")

        for i in range(int(world_count)):
            root = self._world_root_path(i)
            UsdGeom.Xform.Define(self.stage, root)

            # place root in grid (chocolate layout)
            txyz = self._world_translation(i)
            self._set_root_translation(root, txyz)

            bounds = LocalBounds(
                width_m=float(bounds_size_m),
                length_m=float(bounds_size_m),
                origin_xy=(0.0, 0.0),
            )

            builder = WaymoJsonMiniWorldBuilder(
                stage=self.stage,
                world_root=root,
                bounds=bounds,
                origin_mode=self.origin_mode,
            )

            json_path = json_list[i % len(json_list)]
            builder.build_from_json(
                json_path,
                max_agents=max_agents_per_world,

                # road
                polyline_reduction_area=polyline_reduction_area,
                min_points_for_reduction=min_points_for_reduction,
                jump_break_m=jump_break_m,
                seg_width=seg_width,
                seg_height=seg_height,
                z_lift=z_lift,

                # agents
                spawn_z_m=spawn_z_m,
                goal_radius_m=goal_radius_m,

                # parked cars
                parked_if_start_in_goal=parked_if_start_in_goal,
                start_goal_thresh_m=start_goal_thresh_m,
                parked_ground_z_m=parked_ground_z_m,
                parked_chassis_size_m=parked_chassis_size_m,
                parked_wheel_radius_m=parked_wheel_radius_m,
                parked_wheel_thickness_m=parked_wheel_thickness_m,
                parked_wheel_inset_x_m=parked_wheel_inset_x_m,
                parked_wheel_inset_y_m=parked_wheel_inset_y_m,
                parked_ground_clearance_m=parked_ground_clearance_m,

                # goal ring
                goal_ring_z_m=goal_ring_z_m,
                goal_ring_tube_radius_m=goal_ring_tube_radius_m,
                goal_trigger_height_m=goal_trigger_height_m,
            )

        print(f"[ChocolateBarConstructor] Built {world_count} worlds under {self.root_container}")


# ----------------------------
# Example usage (your style)
# ----------------------------



In [ ]:

from pathlib import Path
import omni.usd

stage = omni.usd.get_context().get_stage()

scene_dir = Path("/home/yz8733/Github/isaac-rl/data/processed/waymo_scenes_json")
json_paths = sorted(scene_dir.glob("scene_*.json"))[:16]

layout = GridLayout(world_size_m=(200.0, 200.0), padding_m=0.0, grid_cols=10, base_z_m=0.0)

ctor = ChocolateBarConstructor(
    stage=stage,
    root_container="/World/MiniWorlds",
    layout=layout,
    origin_mode="center",
)

ctor.build(
    json_paths=json_paths,
    world_count=100,
    bounds_size_m=200.0,
    max_agents_per_world=30,

    # road
    jump_break_m=3.0,
    seg_width=0.10,
    seg_height=0.10,
    z_lift=0.02,
    polyline_reduction_area=0.0,

    # agents
    spawn_z_m=1.0,
    goal_radius_m=3.0,

    # start_in_goal => parked car (NO goal)
    parked_if_start_in_goal=True,
    start_goal_thresh_m=10,            # None => uses goal_radius_m
    parked_ground_z_m=0.0,
    parked_chassis_size_m=(4.0, 2.0, 1.0),
    parked_wheel_radius_m=0.35,
    parked_wheel_thickness_m=0.15,
    parked_wheel_inset_x_m=0.6,
    parked_wheel_inset_y_m=0.05,
    parked_ground_clearance_m=0.25,

    # goal ring enforced on ground
    goal_ring_z_m=0.0,
    goal_ring_tube_radius_m=0.12,
    goal_trigger_height_m=0.6,
)


[MiniWorldBuilder] Agents kept=2 skipped=7
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=9 skipped=5
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=18 skipped=2
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=30 skipped=3
[MiniWorldBuilder] Agents kept=13 skipped=1
[MiniWorldBuilder] Agents kept=28 skipped=2
[MiniWorldBuilder] Agents kept=0 skipped=4
[MiniWorldBuilder] Agents kept=0 skipped=17
[MiniWorldBuilder] Agents kept=11 skipped=13
[MiniWorldBuilder] Agents kept=8 skipped=1
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=19 skipped=6
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=2 skipped=7
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=9 skipped=5
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=18 skipped=2
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=30 ski

### After building, get the controllers

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple, Iterable, Union

import numpy as np
import omni.usd
from pxr import Usd, UsdGeom, Sdf


# Your known working attr names:
ACCEL_ATTR = "physxVehicleController:accelerator"
STEER_ATTR = "physxVehicleController:steer"
BRAKE_ATTR = "physxVehicleController:brake0"


@dataclass(frozen=True)
class AgentKey:
    world_idx: int
    agent_id: int


@dataclass
class AgentHandle:
    key: AgentKey
    vehicle_root_path: str          # .../Vehicle_Parent/Vehicle
    ctrl_path: str                  # vehicle_root_path + ctrl_suffix
    pose_path: str                  # usually vehicle_root_path (stable)
    ctrl_prim: Usd.Prim
    pose_prim: Usd.Prim
    xform: UsdGeom.Xformable
    accel: Usd.Attribute
    steer: Optional[Usd.Attribute]
    brake: Optional[Usd.Attribute]


class ChocolateWorldVehicleController:
    """
    Clean controller for ChocolateBarConstructor worlds.

    - Enumerates vehicles by walking ONLY the expected world roots:
        /World/MiniWorlds/world_000 .. world_{world_count-1:03d}
      and ONLY looking at direct agent prim children in /Agents.

    - No "poke and find out" of arbitrary schema/attrs.
      The ONLY configurable bit is ctrl_suffix that defines where the controller attrs live.

    Usage:
      ctrl = ChocolateWorldVehicleController(stage, world_count=100, ctrl_suffix="/VehicleController")
      ctrl.refresh()

      # batch (aligned to ctrl.keys()):
      A = np.zeros((len(ctrl.keys()), 3), np.float32)
      A[:,0] = 0.2  # throttle
      ctrl.apply_all(A)

      # individual:
      ctrl.apply_one(99, 3163, thr=0.3, steer=0.0, brake=0.0)
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        *,
        root_container: str = "/World/MiniWorlds",
        world_prefix: str = "world_",
        world_count: int = 100,
        ctrl_suffix: str = "",  # "" OR "/VehicleController" (set once, no guessing)
        accel_attr: str = ACCEL_ATTR,
        steer_attr: str = STEER_ATTR,
        brake_attr: str = BRAKE_ATTR,
        verbose: bool = True,
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.root_container = root_container
        self.world_prefix = world_prefix
        self.world_count = int(world_count)

        self.ctrl_suffix = str(ctrl_suffix)
        self.accel_attr = accel_attr
        self.steer_attr = steer_attr
        self.brake_attr = brake_attr
        self.verbose = verbose

        self._handles: Dict[AgentKey, AgentHandle] = {}
        self._ordered_keys: List[AgentKey] = []

    # -------------------------
    # Build registry
    # -------------------------

    def refresh(self) -> None:
        self._handles.clear()

        found_agents = 0
        found_ctrl = 0

        for wi in range(self.world_count):
            world_root = f"{self.root_container}/{self.world_prefix}{wi:03d}"
            agents_root = f"{world_root}/Agents"
            agents_prim = self.stage.GetPrimAtPath(agents_root)
            if not agents_prim.IsValid():
                continue

            # Only iterate direct children: Agent_####_id####
            for agent_prim in agents_prim.GetAllChildren():
                name = agent_prim.GetName()
                agent_id = self._parse_agent_id_from_agent_name(name)
                if agent_id is None:
                    continue

                # Vehicle root is fixed by your builder:
                vehicle_root_path = f"{agent_prim.GetPath().pathString}/Vehicle_Parent/Vehicle"
                vehicle_root_prim = self.stage.GetPrimAtPath(vehicle_root_path)
                if not vehicle_root_prim.IsValid():
                    continue

                found_agents += 1

                # Controller prim path is fixed by ctrl_suffix (no scanning)
                ctrl_path = vehicle_root_path + self.ctrl_suffix
                ctrl_prim = self.stage.GetPrimAtPath(ctrl_path)
                if not ctrl_prim.IsValid():
                    continue

                accel = ctrl_prim.GetAttribute(self.accel_attr)
                if not accel.IsValid():
                    continue

                steer = ctrl_prim.GetAttribute(self.steer_attr)
                brake = ctrl_prim.GetAttribute(self.brake_attr)

                key = AgentKey(world_idx=wi, agent_id=int(agent_id))
                handle = AgentHandle(
                    key=key,
                    vehicle_root_path=vehicle_root_path,
                    ctrl_path=ctrl_path,
                    pose_path=vehicle_root_path,  # stable pose prim
                    ctrl_prim=ctrl_prim,
                    pose_prim=vehicle_root_prim,
                    xform=UsdGeom.Xformable(vehicle_root_prim),
                    accel=accel,
                    steer=steer if steer.IsValid() else None,
                    brake=brake if brake.IsValid() else None,
                )

                self._handles[key] = handle
                found_ctrl += 1

        self._ordered_keys = sorted(self._handles.keys(), key=lambda k: (k.world_idx, k.agent_id))

        if self.verbose:
            print(f"[ChocoCtrl] refresh: agents_with_vehicle={found_agents}, controllable={found_ctrl}, keys={len(self._ordered_keys)}")
            if len(self._ordered_keys) == 0:
                print(f"[ChocoCtrl] NOTE: controllable=0. Check ctrl_suffix='{self.ctrl_suffix}' and attr names.")

    def keys(self) -> List[AgentKey]:
        return list(self._ordered_keys)

    def get(self, world_idx: int, agent_id: int) -> Optional[AgentHandle]:
        return self._handles.get(AgentKey(int(world_idx), int(agent_id)))

    # -------------------------
    # Apply controls
    # -------------------------

    def apply_one(self, world_idx: int, agent_id: int, *, thr: float, steer: float, brake: float) -> bool:
        h = self.get(world_idx, agent_id)
        if h is None:
            return False
        return self._apply_handle(h, thr, steer, brake)

    def apply_batch(self, keys: Iterable[AgentKey], controls: Union[np.ndarray, List[List[float]]]) -> int:
        """
        controls: (K,3) [thr, steer, brake] aligned with keys iteration order
        """
        Klist = list(keys)
        U = np.asarray(controls, dtype=np.float32)
        if U.ndim != 2 or U.shape[0] != len(Klist) or U.shape[1] != 3:
            raise ValueError(f"controls must be (K,3). got {U.shape} for K={len(Klist)}")

        ok = 0
        for i, k in enumerate(Klist):
            h = self._handles.get(k)
            if h is None:
                continue
            if self._apply_handle(h, float(U[i,0]), float(U[i,1]), float(U[i,2])):
                ok += 1
        return ok

    def apply_all(self, controls: Union[np.ndarray, List[List[float]]]) -> int:
        """Apply to all controllable vehicles in keys() order."""
        return self.apply_batch(self._ordered_keys, controls)

    def zero_all(self, brake: float = 1.0) -> int:
        Ks = self._ordered_keys
        if not Ks:
            return 0
        U = np.zeros((len(Ks), 3), dtype=np.float32)
        U[:, 2] = float(brake)
        return self.apply_all(U)

    # -------------------------
    # Pose helpers
    # -------------------------

    def get_xy_u(self, world_idx: int, agent_id: int) -> Optional[Tuple[float, float]]:
        h = self.get(world_idx, agent_id)
        if h is None:
            return None
        xf = h.xform.ComputeLocalToWorldTransform(Usd.TimeCode.Default())
        p = xf.ExtractTranslation()
        return (float(p[0]), float(p[1]))

    # -------------------------
    # Internals
    # -------------------------

    def _apply_handle(self, h: AgentHandle, thr: float, steer: float, brake: float) -> bool:
        try:
            h.accel.Set(float(thr))
            if h.steer is not None:
                h.steer.Set(float(steer))
            if h.brake is not None:
                h.brake.Set(float(brake))
            return True
        except Exception:
            return False

    @staticmethod
    def _parse_agent_id_from_agent_name(name: str) -> Optional[int]:
        # expects Agent_0028_id3163
        s = str(name)
        j = s.rfind("_id")
        if j < 0:
            return None
        digits = []
        for ch in s[j+3:]:
            if ch.isdigit():
                digits.append(ch)
            else:
                break
        if not digits:
            return None
        try:
            return int("".join(digits))
        except Exception:
            return None


In [ ]:
ctrl = ChocolateWorldVehicleController(
    stage=stage,
    root_container="/World/MiniWorlds",
    world_count=100,

    # IMPORTANT: set this ONCE based on your known working controller prim location:
    # If your attrs live directly on .../Vehicle_Parent/Vehicle, keep ""
    # If they live under .../Vehicle_Parent/Vehicle/VehicleController, set "/VehicleController"
    ctrl_suffix="/Vehicle",  # or "/VehicleController"

    verbose=True,
)
ctrl.refresh()
print(ctrl.keys()[:10])


[ChocoCtrl] refresh: agents_with_vehicle=737, controllable=737, keys=737
[AgentKey(world_idx=0, agent_id=192), AgentKey(world_idx=0, agent_id=194), AgentKey(world_idx=1, agent_id=2036), AgentKey(world_idx=1, agent_id=4784), AgentKey(world_idx=2, agent_id=38), AgentKey(world_idx=2, agent_id=40), AgentKey(world_idx=2, agent_id=44), AgentKey(world_idx=2, agent_id=63), AgentKey(world_idx=2, agent_id=66), AgentKey(world_idx=2, agent_id=68)]


### Cone builder

In [ ]:
from __future__ import annotations

import math
import random
from dataclasses import dataclass
from typing import List, Optional, Tuple

from pxr import Gf, Sdf, Usd, UsdGeom, UsdPhysics
import omni.usd


def _meters_per_unit(stage: Usd.Stage) -> float:
    mpu = UsdGeom.GetStageMetersPerUnit(stage)
    return float(mpu) if mpu and float(mpu) > 0 else 0.01


def _yaw_from_quath_z_only(q: Gf.Quath) -> float:
    w = float(q.GetReal())
    z = float(q.GetImaginary()[2])
    return 2.0 * math.atan2(z, w)


def _dist2_xy(a: Gf.Vec3f, b: Gf.Vec3f) -> float:
    dx = float(a[0] - b[0])
    dy = float(a[1] - b[1])
    return dx * dx + dy * dy


@dataclass
class ConeConfig:
    radius_m: float = 0.18
    height_m: float = 0.40
    density_kg_m3: float = 350.0
    enable_gravity: bool = True
    color_rgb: Tuple[float, float, float] = (1.0, 0.45, 0.05)


class WorkZoneOnInstancedSegments:
    """
    - Pick one Type_02 PointInstancer (deterministic via seed)
    - Spawn cones on a CONTINUOUS SUBSET of its instances (a run), not the whole road
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        *,
        world_root: str,
        cone_cfg: ConeConfig = ConeConfig(),
        workzones_root_name: str = "WorkZones",
        seed: int = 12345,
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.world_root = world_root.rstrip("/")
        self.cone_cfg = cone_cfg
        self.rng = random.Random(int(seed))
        self.mpu = _meters_per_unit(self.stage)

        self.workzones_root = f"{self.world_root}/{workzones_root_name}"
        UsdGeom.Xform.Define(self.stage, self.workzones_root)

    def clear_workzones(self) -> None:
        try:
            self.stage.RemovePrim(Sdf.Path(self.workzones_root))
        except Exception:
            pass
        UsdGeom.Xform.Define(self.stage, self.workzones_root)

    def _find_type02_instancers(self) -> List[UsdGeom.PointInstancer]:
        type02_root_path = f"{self.world_root}/Road/Type_02"
        root = self.stage.GetPrimAtPath(type02_root_path)
        if not root.IsValid():
            return []
        out: List[UsdGeom.PointInstancer] = []
        stack = [root]
        while stack:
            p = stack.pop()
            if p.GetTypeName() == "PointInstancer":
                out.append(UsdGeom.PointInstancer(p))
            for c in p.GetChildren():
                stack.append(c)
        return out

    # CHANGED: returns Optional[...] and DOES NOT throw when missing
    def pick_random_type02_instancer(self, debug: bool = True) -> Optional[UsdGeom.PointInstancer]:
        insts = self._find_type02_instancers()
        if not insts:
            if debug:
                print(f"[WZ] skip: no PointInstancer under {self.world_root}/Road/Type_02")
            return None

        inst = self.rng.choice(insts)
        if debug:
            try:
                n = len(inst.GetPositionsAttr().Get() or [])
            except Exception:
                n = -1
            print(f"[WZ] picked instancer: {inst.GetPath().pathString} (instances={n})")
        return inst

    def _spawn_one_cone(
        self,
        cone_path: str,
        *,
        center_units: Gf.Vec3f,
        yaw_deg: float,
        z_override_m: Optional[float] = 0.0,
        z_lift_m: float = 0.0,
    ) -> None:
        stage = self.stage
        mpu = self.mpu

        cx_m = float(center_units[0]) * mpu
        cy_m = float(center_units[1]) * mpu
        cz_m = float(center_units[2]) * mpu

        if z_override_m is not None:
            cz_m = float(z_override_m)
        cz_m += float(z_lift_m)

        x_u, y_u, z_u = (cx_m / mpu, cy_m / mpu, cz_m / mpu)

        cone = UsdGeom.Cone.Define(stage, cone_path)
        cone.CreateRadiusAttr(float(self.cone_cfg.radius_m / mpu))
        cone.CreateHeightAttr(float(self.cone_cfg.height_m / mpu))

        xapi = UsdGeom.XformCommonAPI(cone)
        xapi.SetTranslate(Gf.Vec3d(x_u, y_u, z_u))
        xapi.SetRotate(Gf.Vec3f(0.0, 0.0, float(yaw_deg)), UsdGeom.XformCommonAPI.RotationOrderXYZ)

        try:
            g = UsdGeom.Gprim(cone.GetPrim())
            g.CreateDisplayColorAttr([Gf.Vec3f(*map(float, self.cone_cfg.color_rgb))])
        except Exception:
            pass

        prim = cone.GetPrim()
        UsdPhysics.RigidBodyAPI.Apply(prim)
        UsdPhysics.CollisionAPI.Apply(prim)

        try:
            mass_api = UsdPhysics.MassAPI.Apply(prim)
            mass_api.CreateDensityAttr(float(self.cone_cfg.density_kg_m3))
        except Exception:
            pass

        if not self.cone_cfg.enable_gravity:
            try:
                import pxr.PhysxSchema as PhysxSchema  # type: ignore
                rb = PhysxSchema.PhysxRigidBodyAPI.Apply(prim)
                rb.CreateDisableGravityAttr(True)
            except Exception:
                pass

    def _pick_continuous_run_by_meters(
        self,
        positions: List[Gf.Vec3f],
        *,
        run_len_m: float,
        stride: int,
        max_gap_m: float = 6.0,
        debug: bool = True,
    ) -> Tuple[int, int]:
        n = len(positions)
        if n < 2:
            raise RuntimeError("[WZ] not enough instances to form a run")

        stride = max(1, int(stride))
        run_len_m = max(0.0, float(run_len_m))
        max_gap_u2 = (float(max_gap_m) / self.mpu) ** 2

        start = self.rng.randint(0, n - 2)
        traveled_m = 0.0
        end = start + 1

        prev = positions[start]
        while end < n:
            cur = positions[end]
            d2 = _dist2_xy(prev, cur)
            if d2 > max_gap_u2:
                break

            traveled_m += math.sqrt(d2) * self.mpu

            if traveled_m >= run_len_m:
                end += 1
                break

            prev = cur
            end += 1

        if end <= start + 1:
            end = min(n, start + 2)

        if debug:
            print(f"[WZ run] start={start} end={end} traveled_m~{traveled_m:.2f} (target {run_len_m:.2f})")
        return start, end

    # CHANGED: return Optional[str] and skip cleanly when missing/empty
    def spawn_cones_on_continuous_subset(
        self,
        *,
        name: str = "WZ_000",
        instancer: Optional[UsdGeom.PointInstancer] = None,
        stride: int = 2,
        run_len_m: float = 30.0,
        max_gap_m: float = 6.0,
        max_cones: Optional[int] = None,
        z_override_m: Optional[float] = 0.0,
        z_lift_m: float = 0.02,
        debug: bool = True,
    ) -> Optional[str]:
        if instancer is None:
            instancer = self.pick_random_type02_instancer(debug=debug)
            if instancer is None:
                # no Type_02 instancer in this world
                return None

        stride = max(1, int(stride))

        pos = instancer.GetPositionsAttr().Get() or []
        ori = instancer.GetOrientationsAttr().Get() or []
        if len(pos) == 0 or len(ori) == 0:
            if debug:
                print(f"[WZ] skip: instancer has empty positions/orientations: {instancer.GetPath().pathString}")
            return None

        n = min(len(pos), len(ori))
        pos = list(pos[:n])
        ori = list(ori[:n])

        try:
            start, end = self._pick_continuous_run_by_meters(
                pos,
                run_len_m=float(run_len_m),
                stride=stride,
                max_gap_m=float(max_gap_m),
                debug=debug,
            )
        except Exception as e:
            if debug:
                print(f"[WZ] skip: failed to pick run ({e}) for {instancer.GetPath().pathString}")
            return None

        idxs = list(range(start, end, stride))
        if max_cones is not None:
            idxs = idxs[: int(max_cones)]

        wz_root = f"{self.workzones_root}/{name}"
        UsdGeom.Xform.Define(self.stage, wz_root)

        if debug:
            print(
                f"[WZ] cones on subset: inst_n={n} run=[{start},{end}) stride={stride} cones={len(idxs)} "
                f"(run_len_m={run_len_m}, max_gap_m={max_gap_m})"
            )

        for k, j in enumerate(idxs):
            yaw = _yaw_from_quath_z_only(ori[j])
            yaw_deg = float(math.degrees(yaw))
            self._spawn_one_cone(
                f"{wz_root}/Cone_{k:05d}",
                center_units=pos[j],
                yaw_deg=yaw_deg,
                z_override_m=z_override_m,
                z_lift_m=z_lift_m,
            )

        wz_prim = self.stage.GetPrimAtPath(wz_root)
        wz_prim.SetCustomDataByKey("is_workzone", True)
        wz_prim.SetCustomDataByKey("road_type", 2)
        wz_prim.SetCustomDataByKey("source_instancer", instancer.GetPath().pathString)
        wz_prim.SetCustomDataByKey("subset_start", int(start))
        wz_prim.SetCustomDataByKey("subset_end", int(end))
        wz_prim.SetCustomDataByKey("stride", int(stride))
        wz_prim.SetCustomDataByKey("cone_count", int(len(idxs)))
        wz_prim.SetCustomDataByKey("run_len_m", float(run_len_m))
        wz_prim.SetCustomDataByKey("max_gap_m", float(max_gap_m))

        return wz_root


In [ ]:
wz = WorkZoneOnInstancedSegments(world_root="/World/MiniWorlds/world_044", seed=45)
inst = wz.pick_random_type02_instancer()
wz.spawn_cones_on_continuous_subset(
    name="WZ_subset",
    instancer=inst,
    run_len_m=40.0,
    stride=3,
    max_gap_m=8.0,
    z_override_m=0.0,
    z_lift_m=0.02,
)


[WZ] picked instancer: /World/MiniWorlds/world_044/Road/Type_02/Segments (instances=4618)
[WZ run] start=3421 end=3463 traveled_m~40.87 (target 40.00)
[WZ] cones on subset: inst_n=4618 run=[3421,3463) stride=3 cones=14 (run_len_m=40.0, max_gap_m=8.0)

In [ ]:
BASE_SEED = 45

for wi in range(100):
    wz = WorkZoneOnInstancedSegments(
        world_root=f"/World/MiniWorlds/world_{wi:03d}",
        seed=BASE_SEED + wi,
    )

    out = wz.spawn_cones_on_continuous_subset(
        name=f"WZ_subset_{wi:03d}",
        run_len_m=40.0,
        stride=3,
        max_gap_m=8.0,
        z_override_m=0.0,
        z_lift_m=0.02,
        debug=True,
    )

    # out is None when skipped
    if out is None:
        continue


[WZ] picked instancer: /World/MiniWorlds/world_000/Road/Type_02/Segments (instances=3282)
[WZ run] start=1710 end=1715 traveled_m~3.99 (target 40.00)
[WZ] cones on subset: inst_n=3282 run=[1710,1715) stride=3 cones=2 (run_len_m=40.0, max_gap_m=8.0)
[WZ] picked instancer: /World/MiniWorlds/world_001/Road/Type_02/Segments (instances=1448)
[WZ run] start=819 end=821 traveled_m~1.00 (target 40.00)
[WZ] cones on subset: inst_n=1448 run=[819,821) stride=3 cones=1 (run_len_m=40.0, max_gap_m=8.0)
[WZ] skip: no PointInstancer under /World/MiniWorlds/world_002/Road/Type_02
[WZ] picked instancer: /World/MiniWorlds/world_003/Road/Type_02/Segments (instances=4176)
[WZ run] start=1080 end=1089 traveled_m~7.52 (target 40.00)
[WZ] cones on subset: inst_n=4176 run=[1080,1089) stride=3 cones=3 (run_len_m=40.0, max_gap_m=8.0)
[WZ] picked instancer: /World/MiniWorlds/world_004/Road/Type_02/Segments (instances=1033)
[WZ run] start=705 end=715 traveled_m~8.70 (target 40.00)
[WZ] cones on subset: inst_n=1033

[WZ pick] Type_02 ok: inst=/World/MiniWorlds/world_044/Road/Type_02/Segments idx=83 len_m=0.998 center_m=(-44.05,-69.18,0.25) yaw_deg=-161.3
[WZ spawn] 15 cones: cover_len_m=35.0, spacing_m=2.5, lat_off_m=1.2, start_off_m=0.0 (Type_02 only)

[WZ pick] ok: inst=/World/MiniWorlds/world_044/Road/Type_07/Segments idx=109 len_m=0.989 center_m=(9.98,-15.97,-0.57) yaw_deg=2.2
[WZ spawn] 15 cones: cover_len_m=35.0, spacing_m=2.5, offset_lat_m=1.2, start_offset_m=0.0

### Loop for the simulator 

In [ ]:
import omni.kit.app
import omni.timeline

# Global handle so we can stop it from another cell
_SIM_TICK_SUB = globals().get("_SIM_TICK_SUB", None)

# If already running, stop old subscription first
if _SIM_TICK_SUB is not None:
    try:
        _SIM_TICK_SUB = None
    except Exception:
        pass

app = omni.kit.app.get_app()
timeline = omni.timeline.get_timeline_interface()

def _tick_noop(e):
    # Intentionally do nothing; this callback is just to "tick"
    # (Kit will call it every frame while the app is running)
    return

# Start sim
timeline.play()

# Install per-frame callback (keeps "ticking" without blocking the notebook cell)
_SIM_TICK_SUB = app.get_update_event_stream().create_subscription_to_pop(_tick_noop)

globals()["_SIM_TICK_SUB"] = _SIM_TICK_SUB
print("[sim] started (non-blocking). Run the STOP cell to stop.")


[sim] started (non-blocking). Run the STOP cell to stop.

In [ ]:
import omni.timeline

_SIM_TICK_SUB = globals().get("_SIM_TICK_SUB", None)
globals()["_SIM_TICK_SUB"] = None
_SIM_TICK_SUB = None  # drop subscription reference => unsubscribes

omni.timeline.get_timeline_interface().stop()
print("[sim] stopped")


[sim] stopped

### Camera code

# Working camera for demo filming

In [ ]:
import math
from pathlib import Path

import omni.kit.app
import omni.timeline
import omni.usd

from pxr import Usd, UsdGeom, Gf

import numpy as np  # <-- make np global so you can use it at any tick

# ----------------------------
# USER SETTINGS
# ----------------------------
OUT_DIR = Path("/home/yz8733/Github/isaac-rl/_captures_300")
OUT_DIR.mkdir(parents=True, exist_ok=True)

START_POS_UNITS = Gf.Vec3d(831.0763414039035, 760.5554554448777, 25.36115399004309)
ROT_AXIS = Gf.Vec3d(0.8568526052619418, -0.2815100301095792, -0.4319209601345214)
ROT_ANGLE_DEG = 74.5166892795625

TICKS_STILL = 100
TICKS_BACK  = 1400
TOTAL_TICKS = TICKS_STILL + TICKS_BACK

BACKWARD_SPEED_M_PER_TICK = 1.0
CAM_PATH = "/World/CaptureCam"

# ----------------------------
# INTERNALS
# ----------------------------
stage = omni.usd.get_context().get_stage()
mpu = float(UsdGeom.GetStageMetersPerUnit(stage) or 0.01)
base_speed_units_per_tick = float(BACKWARD_SPEED_M_PER_TICK / mpu)

app = omni.kit.app.get_app()
timeline = omni.timeline.get_timeline_interface()

# Stop any previous runner
_CAP_RUN_SUB = globals().get("_CAP_RUN_SUB", None)
if _CAP_RUN_SUB is not None:
    try:
        if hasattr(_CAP_RUN_SUB, "unsubscribe"):
            _CAP_RUN_SUB.unsubscribe()
        elif hasattr(_CAP_RUN_SUB, "destroy"):
            _CAP_RUN_SUB.destroy()
    except Exception:
        pass
    globals()["_CAP_RUN_SUB"] = None
    _CAP_RUN_SUB = None


def _set_viewport_camera(cam_path: str):
    try:
        import omni.kit.viewport.utility as vutil
        vw = vutil.get_active_viewport_window()
        if vw is None:
            print("[cam] No active viewport window.")
            return
        vw.viewport_api.set_active_camera(cam_path)
        print("[cam] viewport camera set to", cam_path)
    except Exception as e:
        print("[cam] failed to set viewport camera:", e)


def _capture_active_viewport_png(filepath: str) -> bool:
    try:
        import omni.kit.viewport.utility as vutil
        vw = vutil.get_active_viewport_window()
        if vw is None:
            return False
        vp = vw.viewport_api
        if hasattr(vutil, "capture_viewport_to_file"):
            vutil.capture_viewport_to_file(vp, filepath)
            return True
        if hasattr(vutil, "capture_viewport_to_file_async"):
            vutil.capture_viewport_to_file_async(vp, filepath)
            return True
    except Exception:
        pass

    try:
        import omni.kit.capture.viewport as cap
        cap.capture_viewport_to_file(filepath)
        return True
    except Exception:
        pass

    return False


def _ensure_camera(cam_path: str) -> UsdGeom.Camera:
    cam = UsdGeom.Camera.Define(stage, cam_path)
    cam.CreateFocalLengthAttr(24.0)
    cam.CreateHorizontalApertureAttr(20.955)
    cam.CreateVerticalApertureAttr(15.2908)
    return cam


def _set_camera_world_matrix(cam_prim: Usd.Prim, M_world_units: Gf.Matrix4d):
    xf = UsdGeom.Xformable(cam_prim)
    ops = xf.GetOrderedXformOps()
    op = ops[0] if ops else xf.AddTransformOp()
    op.Set(M_world_units)


def _make_start_matrix() -> Gf.Matrix4d:
    R = Gf.Rotation(ROT_AXIS, float(ROT_ANGLE_DEG))
    M = Gf.Matrix4d(1.0)
    M.SetRotate(R)
    M.SetTranslateOnly(START_POS_UNITS)
    return M


def _normalize(v: Gf.Vec3d) -> Gf.Vec3d:
    n = math.sqrt(v[0]*v[0] + v[1]*v[1] + v[2]*v[2])
    if n < 1e-8:
        return Gf.Vec3d(0.0, 0.0, 0.0)
    return Gf.Vec3d(v[0]/n, v[1]/n, v[2]/n)


def _backward_dir_units_from_matrix(M: Gf.Matrix4d) -> Gf.Vec3d:
    origin_world = M.Transform(Gf.Vec3d(0.0, 0.0, 0.0))
    front_world  = M.Transform(Gf.Vec3d(0.0, 0.0, -1.0))
    fwd = _normalize(Gf.Vec3d(front_world[0] - origin_world[0],
                             front_world[1] - origin_world[1],
                             front_world[2] - origin_world[2]))
    back = Gf.Vec3d(-fwd[0], -fwd[1], -fwd[2])
    return _normalize(back)


# Create & place camera
cam = _ensure_camera(CAM_PATH)
cam_prim = cam.GetPrim()

M0 = _make_start_matrix()
_set_camera_world_matrix(cam_prim, M0)
_set_viewport_camera(CAM_PATH)

back_dir0 = _backward_dir_units_from_matrix(M0)
print("[cam] back_dir(world units):", (float(back_dir0[0]), float(back_dir0[1]), float(back_dir0[2])))

# State
_state = {
    "tick": 0,
    "M": M0,
    "back_dir": back_dir0,
    "capture_ok": None,
    "Ks": None,                      # store keys once
    "speed_units_per_tick": base_speed_units_per_tick,  # <-- speed lives here
}

def _runner_tick(e):
    if not timeline.is_playing():
        return

    t = _state["tick"]

    # --- tick 0 init ---
    if t == 0:
        Ks = ctrl.keys()
        _state["Ks"] = Ks

        U = np.zeros((len(Ks), 3), dtype=np.float32)
        U[:, 0] = 1.0
        U[:, 1] = 0.0
        U[:, 2] = 0.0
        ctrl.apply_all(U)
        print(f"[ctrl] applied throttle=1.0 to {len(Ks)} agents at tick 0")

        # Manual tweaks
        ctrl.apply_one(44, 1287, thr=0.1, steer=0.0, brake=0.0)
        ctrl.apply_one(44, 1276, thr=1.0, steer=1.0, brake=0.0)
        ctrl.apply_one(44, 1277, thr=1.0, steer=1.0, brake=0.0)

    # --- stop ---
    if t >= TOTAL_TICKS:
        timeline.stop()
        try:
            sub = globals().get("_CAP_RUN_SUB", None)
            if sub is not None:
                if hasattr(sub, "unsubscribe"):
                    sub.unsubscribe()
                elif hasattr(sub, "destroy"):
                    sub.destroy()
        except Exception:
            pass
        globals()["_CAP_RUN_SUB"] = None
        print(f"[run] done. saved {TOTAL_TICKS} images to {OUT_DIR}")
        return

    # --- scripted steering changes ---
    if t == 110:
        ctrl.apply_one(44, 1276, thr=1.0, steer=-1.0, brake=0.0)
        ctrl.apply_one(44, 1277, thr=1.0, steer=-1.0, brake=0.0)

    if t == 170:
        ctrl.apply_one(44, 1276, thr=1.0, steer=0.0, brake=0.0)
        ctrl.apply_one(44, 1277, thr=1.0, steer=0.0, brake=0.0)

    if t == 300:
        Ks = _state["Ks"] or ctrl.keys()
        U = np.zeros((len(Ks), 3), dtype=np.float32)
        U[:, 0] = 0.0
        U[:, 1] = 0.0
        U[:, 2] = 1.0
        ctrl.apply_all(U)

        # IMPORTANT: update speed in state (not a local var)
        _state["speed_units_per_tick"] *= 2.0
        print(f"[cam] speed x2 => {_state['speed_units_per_tick']:.3f} units/tick")

    # --- move camera ---
    if t >= TICKS_STILL:
        s = float(_state["speed_units_per_tick"])
        back = _state["back_dir"]
        d = Gf.Vec3d(back[0] * s, back[1] * s, back[2] * s)

        Mcur = _state["M"]
        pos = Mcur.ExtractTranslation()
        pos2 = Gf.Vec3d(pos[0] + d[0], pos[1] + d[1], pos[2] + d[2])
        Mcur.SetTranslateOnly(pos2)

        _state["M"] = Mcur
        _set_camera_world_matrix(cam_prim, Mcur)

    # --- capture ---
    out_path = str(OUT_DIR / f"frame_{t:06d}.png")
    ok = _capture_active_viewport_png(out_path)
    if not ok:
        if _state["capture_ok"] is None:
            _state["capture_ok"] = False
            print("[capture] No supported viewport capture API found. Stopping.")
        timeline.stop()
        globals()["_CAP_RUN_SUB"] = None
        return
    else:
        if _state["capture_ok"] is None:
            _state["capture_ok"] = True
            print("[capture] OK, saving frames...")

    _state["tick"] = t + 1


# subscribe + start
_CAP_RUN_SUB = app.get_update_event_stream().create_subscription_to_pop(_runner_tick)
globals()["_CAP_RUN_SUB"] = _CAP_RUN_SUB

timeline.play()
print(f"[run] started: {TOTAL_TICKS} ticks total ({TICKS_STILL} still + {TICKS_BACK} backward), saving to {OUT_DIR}")


[cam] viewport camera set to /World/CaptureCam
[cam] back_dir(world units): (-0.5425870793645813, -0.7366252517186941, 0.4037109112186396)
[run] started: 1500 ticks total (100 still + 1400 backward), saving to /home/yz8733/Github/isaac-rl/_captures_300

In [ ]:
import math
from pathlib import Path

import omni.kit.app
import omni.timeline
import omni.usd

from pxr import Usd, UsdGeom, Gf

# ----------------------------
# USER SETTINGS
# ----------------------------
OUT_DIR = Path("/home/yz8733/Github/isaac-rl/_captures_300")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Pose you printed (stage units!)
START_POS_UNITS = Gf.Vec3d(831.0763414039035, 760.5554554448777, 25.36115399004309)
ROT_AXIS = Gf.Vec3d(0.8568526052619418, -0.2815100301095792, -0.4319209601345214)
ROT_ANGLE_DEG = 74.5166892795625

TICKS_STILL = 100
TICKS_BACK  = 205
TOTAL_TICKS = TICKS_STILL + TICKS_BACK

# move backward speed (meters per tick)
BACKWARD_SPEED_M_PER_TICK = 1

CAM_PATH = "/World/CaptureCam"

# ----------------------------
# INTERNALS
# ----------------------------
stage = omni.usd.get_context().get_stage()
mpu = float(UsdGeom.GetStageMetersPerUnit(stage) or 0.01)  # meters per stage unit
speed_units_per_tick = float(BACKWARD_SPEED_M_PER_TICK / mpu)

app = omni.kit.app.get_app()
timeline = omni.timeline.get_timeline_interface()

# Stop any previous runner
_CAP_RUN_SUB = globals().get("_CAP_RUN_SUB", None)
if _CAP_RUN_SUB is not None:
    try:
        globals()["_CAP_RUN_SUB"] = None
    except Exception:
        pass
    _CAP_RUN_SUB = None


def _set_viewport_camera(cam_path: str):
    try:
        import omni.kit.viewport.utility as vutil
        vw = vutil.get_active_viewport_window()
        if vw is None:
            print("[cam] No active viewport window.")
            return
        vp = vw.viewport_api
        vp.set_active_camera(cam_path)
        print("[cam] viewport camera set to", cam_path)
    except Exception as e:
        print("[cam] failed to set viewport camera:", e)


def _capture_active_viewport_png(filepath: str) -> bool:
    # Try common viewport capture APIs (varies by Isaac version)
    try:
        import omni.kit.viewport.utility as vutil
        vw = vutil.get_active_viewport_window()
        if vw is None:
            return False
        vp = vw.viewport_api
        if hasattr(vutil, "capture_viewport_to_file"):
            vutil.capture_viewport_to_file(vp, filepath)
            return True
        if hasattr(vutil, "capture_viewport_to_file_async"):
            vutil.capture_viewport_to_file_async(vp, filepath)
            return True
    except Exception:
        pass

    try:
        import omni.kit.capture.viewport as cap
        cap.capture_viewport_to_file(filepath)
        return True
    except Exception:
        pass

    return False


def _ensure_camera(cam_path: str) -> UsdGeom.Camera:
    cam = UsdGeom.Camera.Define(stage, cam_path)
    cam.CreateFocalLengthAttr(24.0)
    cam.CreateHorizontalApertureAttr(20.955)
    cam.CreateVerticalApertureAttr(15.2908)
    return cam


def _set_camera_world_matrix(cam_prim: Usd.Prim, M_world_units: Gf.Matrix4d):
    xf = UsdGeom.Xformable(cam_prim)
    ops = xf.GetOrderedXformOps()
    op = ops[0] if ops else xf.AddTransformOp()
    op.Set(M_world_units)


def _make_start_matrix() -> Gf.Matrix4d:
    R = Gf.Rotation(ROT_AXIS, float(ROT_ANGLE_DEG))
    M = Gf.Matrix4d(1.0)
    M.SetRotate(R)
    M.SetTranslateOnly(START_POS_UNITS)
    return M


def _normalize(v: Gf.Vec3d) -> Gf.Vec3d:
    n = math.sqrt(v[0]*v[0] + v[1]*v[1] + v[2]*v[2])
    if n < 1e-8:
        return Gf.Vec3d(0.0, 0.0, 0.0)
    return Gf.Vec3d(v[0]/n, v[1]/n, v[2]/n)


def _backward_dir_units_from_matrix(M: Gf.Matrix4d) -> Gf.Vec3d:
    """
    Robust: compute camera forward by transforming a local forward point.
    Camera forward is local -Z, so:
      origin_local = (0,0,0)
      front_local  = (0,0,-1)
    world_forward = normalize(front_world - origin_world)
    backward = -world_forward
    """
    origin_world = M.Transform(Gf.Vec3d(0.0, 0.0, 0.0))
    front_world  = M.Transform(Gf.Vec3d(0.0, 0.0, -1.0))
    fwd = _normalize(Gf.Vec3d(front_world[0] - origin_world[0],
                             front_world[1] - origin_world[1],
                             front_world[2] - origin_world[2]))
    back = Gf.Vec3d(-fwd[0], -fwd[1], -fwd[2])
    return _normalize(back)


# Create & place camera
cam = _ensure_camera(CAM_PATH)
cam_prim = cam.GetPrim()

M0 = _make_start_matrix()
_set_camera_world_matrix(cam_prim, M0)
_set_viewport_camera(CAM_PATH)

back_dir0 = _backward_dir_units_from_matrix(M0)
print("[cam] back_dir(world units):", (float(back_dir0[0]), float(back_dir0[1]), float(back_dir0[2])))

# State
_state = {
    "tick": 0,
    "M": M0,
    "back_dir": back_dir0,
    "capture_ok": None,
}

def _runner_tick(e):
    if not timeline.is_playing():
        return

    t = _state["tick"]
    if t == 0:
        import numpy as np
        # assumes you already have `ctrl` in the notebook (your controller object)
        Ks = ctrl.keys()
        U = np.zeros((len(Ks), 3), dtype=np.float32)
        U[:, 0] = 1.0  # throttle
        U[:, 1] = 0.0  # steer
        U[:, 2] = 0.0  # brake
        ctrl.apply_all(U)
        
        print(f"[ctrl] applied throttle=1.0 to {len(Ks)} agents at tick 0")
        #Manual stopping for demo filming
        ctrl.apply_one(44,1287, thr=0.1, steer=0.0, brake=0.0)
        ctrl.apply_one(44,1276, thr=1.0, steer=1.0, brake=0.0)
        ctrl.apply_one(44,1277, thr=1.0, steer=1.0, brake=0.0)
    # stop
    if t >= TOTAL_TICKS:
        timeline.stop()
        globals()["_CAP_RUN_SUB"] = None
        print(f"[run] done. saved {TOTAL_TICKS} images to {OUT_DIR}")
        return
    if t == 110: 
        ctrl.apply_one(44,1276, thr=1.0, steer=-1.0, brake=0.0)
        ctrl.apply_one(44,1277, thr=1.0, steer=-1.0, brake=0.0)
    if t == 170:
        ctrl.apply_one(44,1276, thr=1.0, steer=0.0, brake=0.0)
        ctrl.apply_one(44,1277, thr=1.0, steer=0.0, brake=0.0)  
    if t == 300:
        U = np.zeros((len(Ks), 3), dtype=np.float32)
        U[:, 0] = 0.0  # throttle
        U[:, 1] = 0.0  # steer
        U[:, 2] = 0.0  # brake
        ctrl.apply_all(U)
        speed_units_per_tick = speed_units_per_tick*2
        
    # move backward for last 200 ticks
    if t >= TICKS_STILL:
        back = _state["back_dir"]
        d = Gf.Vec3d(back[0] * speed_units_per_tick,
                     back[1] * speed_units_per_tick,
                     back[2] * speed_units_per_tick)

        Mcur = _state["M"]
        pos = Mcur.ExtractTranslation()
        pos2 = Gf.Vec3d(pos[0] + d[0], pos[1] + d[1], pos[2] + d[2])
        Mcur.SetTranslateOnly(pos2)

        _state["M"] = Mcur
        _set_camera_world_matrix(cam_prim, Mcur)

    # capture
    out_path = str(OUT_DIR / f"frame_{t:06d}.png")
    ok = _capture_active_viewport_png(out_path)

    if not ok:
        if _state["capture_ok"] is None:
            _state["capture_ok"] = False
            print("[capture] No supported viewport capture API found. Stopping.")
        timeline.stop()
        globals()["_CAP_RUN_SUB"] = None
        return
    else:
        if _state["capture_ok"] is None:
            _state["capture_ok"] = True
            print("[capture] OK, saving frames...")

    _state["tick"] = t + 1


# subscribe + start
_CAP_RUN_SUB = app.get_update_event_stream().create_subscription_to_pop(_runner_tick)
globals()["_CAP_RUN_SUB"] = _CAP_RUN_SUB

timeline.play()
print(f"[run] started: {TOTAL_TICKS} ticks total ({TICKS_STILL} still + {TICKS_BACK} backward), saving to {OUT_DIR}")


[cam] viewport camera set to /World/CaptureCam
[cam] back_dir(world units): (-0.5425870793645813, -0.7366252517186941, 0.4037109112186396)
[run] started: 1000 ticks total (100 still + 900 backward), saving to /home/yz8733/Github/isaac-rl/_captures_300

### See where the current camera is 

In [ ]:
import omni.kit.viewport.utility as vutil
import omni.usd
from pxr import Usd, UsdGeom, Gf

stage = omni.usd.get_context().get_stage()

# active viewport
vw = vutil.get_active_viewport_window()
vp = vw.viewport_api

cam_path = vp.get_active_camera()   # string prim path
print("Active viewport camera:", cam_path)

cam_prim = stage.GetPrimAtPath(cam_path)
img = UsdGeom.Imageable(cam_prim)

# World transform
M = img.ComputeLocalToWorldTransform(Usd.TimeCode.Default())
t = M.ExtractTranslation()
r = M.ExtractRotation()  # Gf.Rotation

print("World position (stage units):", (float(t[0]), float(t[1]), float(t[2])))

axis = r.GetAxis()
angle = r.GetAngle()
print("World rotation axis-angle:", (float(axis[0]), float(axis[1]), float(axis[2])), float(angle))
    

Active viewport camera: /OmniverseKit_Persp
World position (stage units): (831.0763414039035, 760.5554554448777, 25.36115399004309)
World rotation axis-angle: (0.8568526052619418, -0.2815100301095792, -0.4319209601345214) 74.5166892795625

# Legacy code\

In [ ]:
# ---- UPDATED: WaymoJsonMiniWorldBuilder (parked cars instead of pushable vehicles) ----

from __future__ import annotations

import json
import math
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional, Sequence, Tuple, Union

import numpy as np
from pxr import Gf, Sdf, Usd, UsdGeom, UsdPhysics, Vt

import omni.usd

# ---- PhysX vehicle wizard imports (THE WORKING ONES you pasted) ----
from omni.physxvehicle.scripts.wizards import physxVehicleWizard as VehicleWizard
from omni.physxvehicle.scripts.helpers.UnitScale import UnitScale
from omni.physxvehicle.scripts.commands import PhysXVehicleWizardCreateCommand

ROOT_PATH = "/World"
SHARED_ROOT = ROOT_PATH + "/VehicleShared"


def _ensure_world_default_prim(stage: Usd.Stage) -> None:
    world = stage.GetPrimAtPath(ROOT_PATH)
    if not world.IsValid():
        world = UsdGeom.Xform.Define(stage, ROOT_PATH).GetPrim()
    if not stage.GetDefaultPrim().IsValid():
        stage.SetDefaultPrim(world)


def _get_unit_scale(stage: Usd.Stage) -> Tuple[UnitScale, float]:
    """Return (UnitScale, meters_per_unit)."""
    meters_per_unit = UsdGeom.GetStageMetersPerUnit(stage)
    if meters_per_unit == 0:
        meters_per_unit = 0.01  # Isaac default often cm
    length_scale = 1.0 / meters_per_unit

    kilograms_per_unit = UsdPhysics.GetStageKilogramsPerUnit(stage)
    if kilograms_per_unit == 0:
        kilograms_per_unit = 1.0
    mass_scale = 1.0 / kilograms_per_unit
    return UnitScale(length_scale, mass_scale), meters_per_unit


def _meters_per_unit(stage: Usd.Stage) -> float:
    mpu = UsdGeom.GetStageMetersPerUnit(stage)
    return float(mpu) if mpu and float(mpu) > 0 else 0.01


def _safe_int(x, default=-1) -> int:
    try:
        return int(x)
    except Exception:
        try:
            return int(float(x))
        except Exception:
            return default


def _safe_float(x, default=None) -> Optional[float]:
    try:
        return float(x)
    except Exception:
        return default


def _quath_from_yaw_z(yaw_rad: float) -> Gf.Quath:
    """Half-precision quat rotation about +Z."""
    half = 0.5 * float(yaw_rad)
    w = float(np.cos(half))
    z = float(np.sin(half))
    try:
        return Gf.Quath(Gf.Half(w), Gf.Half(0.0), Gf.Half(0.0), Gf.Half(z))
    except Exception:
        return Gf.Quath(w, 0.0, 0.0, z)


def _reduce_polyline_xy(points_xyz: np.ndarray, area_thresh: float) -> np.ndarray:
    """Iteratively remove points with small local triangle area (XY) under threshold."""
    pts = np.asarray(points_xyz, dtype=np.float32)
    if pts.shape[0] < 3 or area_thresh <= 0:
        return pts

    keep = np.ones((pts.shape[0],), dtype=bool)
    changed = True

    def tri_area_xy(a, b, c) -> float:
        abx = float(b[0] - a[0]); aby = float(b[1] - a[1])
        acx = float(c[0] - a[0]); acy = float(c[1] - a[1])
        cross = abx * acy - aby * acx
        return 0.5 * abs(cross)

    while changed:
        changed = False
        idxs = np.nonzero(keep)[0]
        if idxs.size < 3:
            break
        for k in range(1, idxs.size - 1):
            i0 = idxs[k - 1]; i1 = idxs[k]; i2 = idxs[k + 1]
            if tri_area_xy(pts[i0], pts[i1], pts[i2]) < area_thresh:
                keep[i1] = False
                changed = True
    return pts[keep]
    
# ----------------------------
# Chocolate-bar multi-world constructor (GRID of mini-worlds)
# ----------------------------

from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Sequence, Tuple, Union

from pxr import Gf, Sdf, Usd, UsdGeom
import omni.usd


@dataclass
class GridLayout:
    world_size_m: Tuple[float, float] = (200.0, 200.0)
    padding_m: float = 0.0
    grid_cols: int = 5
    base_z_m: float = 0.0


class ChocolateBarConstructor:
    """
    Places N mini-world roots in a grid and spawns each from a JSON.
    IMPORTANT: Per-world filtering is LOCAL, grid translation is on the root prim only.
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        root_container: str = "/World/MiniWorlds",
        layout: GridLayout = GridLayout(),
        origin_mode: str = "center",
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.root_container = root_container
        self.layout = layout
        self.origin_mode = origin_mode

        UsdGeom.Xform.Define(self.stage, self.root_container)

    def _world_root_path(self, i: int) -> str:
        return f"{self.root_container}/world_{i:03d}"

    def _world_translation(self, i: int) -> Tuple[float, float, float]:
        cols = int(self.layout.grid_cols)
        sx, sy = self.layout.world_size_m
        pitch_x = float(sx + self.layout.padding_m)
        pitch_y = float(sy + self.layout.padding_m)
        r = i // cols
        c = i % cols
        return (float(c * pitch_x), float(r * pitch_y), float(self.layout.base_z_m))

    def _set_root_translation(self, prim_path: str, txyz: Tuple[float, float, float]) -> None:
        prim = self.stage.GetPrimAtPath(prim_path)
        if not prim.IsValid():
            UsdGeom.Xform.Define(self.stage, prim_path)
            prim = self.stage.GetPrimAtPath(prim_path)
        xform = UsdGeom.XformCommonAPI(prim)
        xform.SetTranslate(Gf.Vec3d(*txyz))

    def clear_all(self) -> None:
        self.stage.RemovePrim(Sdf.Path(self.root_container))
        UsdGeom.Xform.Define(self.stage, self.root_container)

    def build(
        self,
        json_paths: Sequence[Union[str, Path]],
        world_count: int,
        *,
        bounds_size_m: float = 200.0,
        max_agents_per_world: int = 50,

        # road params:
        polyline_reduction_area: float = 0.0,
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,

        # agent params:
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,

        # ---- NEW: parked-car passthrough ----
        parked_if_start_in_goal: bool = True,
        start_goal_thresh_m: Optional[float] = None,
        parked_ground_z_m: float = 0.0,
        parked_chassis_size_m: Tuple[float, float, float] = (4.0, 2.0, 1.0),
        parked_wheel_radius_m: float = 0.35,
        parked_wheel_thickness_m: float = 0.15,
        parked_wheel_inset_x_m: float = 0.35,
        parked_wheel_inset_y_m: float = 0.25,
        parked_ground_clearance_m: float = 0.25,
    ) -> None:
        json_list = [str(Path(p).expanduser().resolve()) for p in json_paths]
        if len(json_list) == 0:
            raise ValueError("json_paths is empty")

        for i in range(int(world_count)):
            root = self._world_root_path(i)
            UsdGeom.Xform.Define(self.stage, root)

            # place root in grid (chocolate layout)
            txyz = self._world_translation(i)
            self._set_root_translation(root, txyz)

            bounds = LocalBounds(
                width_m=float(bounds_size_m),
                length_m=float(bounds_size_m),
                origin_xy=(0.0, 0.0),
            )

            builder = WaymoJsonMiniWorldBuilder(
                stage=self.stage,
                world_root=root,
                bounds=bounds,
                origin_mode=self.origin_mode,
            )

            json_path = json_list[i % len(json_list)]
            builder.build_from_json(
                json_path,
                max_agents=max_agents_per_world,

                # road
                polyline_reduction_area=polyline_reduction_area,
                min_points_for_reduction=min_points_for_reduction,
                jump_break_m=jump_break_m,
                seg_width=seg_width,
                seg_height=seg_height,
                z_lift=z_lift,

                # agents
                spawn_z_m=spawn_z_m,
                goal_radius_m=goal_radius_m,

                # ---- NEW: forwarded parked-car params ----
                parked_if_start_in_goal=parked_if_start_in_goal,
                start_goal_thresh_m=start_goal_thresh_m,
                parked_ground_z_m=parked_ground_z_m,
                parked_chassis_size_m=parked_chassis_size_m,
                parked_wheel_radius_m=parked_wheel_radius_m,
                parked_wheel_thickness_m=parked_wheel_thickness_m,
                parked_wheel_inset_x_m=parked_wheel_inset_x_m,
                parked_wheel_inset_y_m=parked_wheel_inset_y_m,
                parked_ground_clearance_m=parked_ground_clearance_m,
            )

        print(f"[ChocolateBarConstructor] Built {world_count} worlds under {self.root_container}")


@dataclass
class LocalBounds:
    width_m: float = 200.0
    length_m: float = 200.0
    origin_xy: Tuple[float, float] = (0.0, 0.0)


class WaymoJsonMiniWorldBuilder:
    """
    Builds ONE mini-world under world_root.

    NEW behavior:
    - start_in_goal agents become PARKED CARS:
        * static collider on chassis
        * visual wheels only (no wheel colliders)
        * NO destination cylinder spawned
    - other agents remain full PhysX vehicles (wizard) + goal cylinder
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        world_root: str = "/World/MiniWorlds/world_000",
        bounds: Optional[LocalBounds] = None,
        origin_mode: str = "center",  # "center" or "zero"
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.world_root = world_root
        self.bounds = bounds or LocalBounds()
        self.origin_mode = origin_mode

        self._scene_center = np.zeros((3,), dtype=np.float32)

        UsdGeom.Xform.Define(self.stage, self.world_root)

        self.road_root = f"{self.world_root}/Road"
        self.agents_root = f"{self.world_root}/Agents"
        self.goals_root = f"{self.world_root}/Goals"
        UsdGeom.Xform.Define(self.stage, self.road_root)
        UsdGeom.Xform.Define(self.stage, self.agents_root)
        UsdGeom.Xform.Define(self.stage, self.goals_root)

    # -------- bounds gating in LOCAL coords --------
    def _in_bounds_xy(self, x: float, y: float) -> bool:
        ox, oy = self.bounds.origin_xy
        hw = 0.5 * float(self.bounds.width_m)
        hl = 0.5 * float(self.bounds.length_m)
        return (ox - hw <= x <= ox + hw) and (oy - hl <= y <= oy + hl)

    def _filter_polyline_points(self, pts: np.ndarray) -> np.ndarray:
        if pts.shape[0] == 0:
            return pts
        m = np.array([self._in_bounds_xy(float(p[0]), float(p[1])) for p in pts], dtype=bool)
        return pts[m]

    # -------- origin shift --------
    def _compute_scene_center_from_road(self, polylines: List[Dict[str, Any]]) -> np.ndarray:
        all_pts = []
        for pl in polylines:
            xyz = pl.get("xyz", None)
            if not xyz:
                continue
            arr = np.asarray(xyz, dtype=np.float32)
            if arr.ndim == 2 and arr.shape[1] >= 2 and arr.shape[0] >= 1:
                if arr.shape[1] == 2:
                    arr = np.concatenate([arr, np.zeros((arr.shape[0], 1), dtype=np.float32)], axis=1)
                all_pts.append(arr[:, :3])
        if not all_pts:
            return np.zeros((3,), dtype=np.float32)
        pts = np.concatenate(all_pts, axis=0)
        return pts.mean(axis=0)

    def _to_local_xyz(self, x: float, y: float, z: float) -> Tuple[float, float, float]:
        return (
            float(x - self._scene_center[0]),
            float(y - self._scene_center[1]),
            float(z - self._scene_center[2]),
        )

    def _is_start_within_goal(
        self, sx: float, sy: float, sz: float, ex: float, ey: float, ez: float, radius_m: float
    ) -> bool:
        dx = float(ex - sx)
        dy = float(ey - sy)
        dz = float(ez - sz)
        return (dx * dx + dy * dy + dz * dz) <= float(radius_m * radius_m)

    # -------- road segments grouped by type --------
    def build_road_segments_grouped(
        self,
        cfg: Dict[str, Any],
        polyline_reduction_area: float = 0.0,
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,
        max_segments_per_type: Optional[int] = None,
    ) -> None:
        road = (cfg.get("road", {}) or {})
        polylines = road.get("polylines", []) or []

        if self.origin_mode == "center":
            self._scene_center = self._compute_scene_center_from_road(polylines)
        else:
            self._scene_center = np.zeros((3,), dtype=np.float32)

        by_type: Dict[int, List[np.ndarray]] = {}
        for pl in polylines:
            t = _safe_int(pl.get("type", -1), -1)
            xyz = pl.get("xyz", None)
            if not xyz:
                continue
            pts = np.asarray(xyz, dtype=np.float32)
            if pts.ndim != 2 or pts.shape[0] < 2:
                continue
            if pts.shape[1] == 2:
                pts = np.concatenate([pts, np.zeros((pts.shape[0], 1), dtype=np.float32)], axis=1)

            pts_local = np.array([self._to_local_xyz(p[0], p[1], p[2]) for p in pts[:, :3]], dtype=np.float32)
            pts_local = self._filter_polyline_points(pts_local)
            if pts_local.shape[0] < 2:
                continue

            if polyline_reduction_area > 0 and pts_local.shape[0] >= int(min_points_for_reduction):
                pts_local = _reduce_polyline_xy(pts_local, float(polyline_reduction_area))
                if pts_local.shape[0] < 2:
                    continue

            by_type.setdefault(t, []).append(pts_local)

        for t, polys in sorted(by_type.items(), key=lambda kv: kv[0]):
            type_root = f"{self.road_root}/Type_{t:02d}"
            instancer_path = f"{type_root}/Segments"
            UsdGeom.Xform.Define(self.stage, type_root)

            seg_positions_py = []
            seg_orients_py = []
            seg_scales_py = []
            proto_indices_py = []

            seg_count = 0
            for poly in polys:
                for i in range(poly.shape[0] - 1):
                    p0 = poly[i]
                    p1 = poly[i + 1]

                    if not (self._in_bounds_xy(float(p0[0]), float(p0[1])) and self._in_bounds_xy(float(p1[0]), float(p1[1]))):
                        continue

                    dx = float(p1[0] - p0[0])
                    dy = float(p1[1] - p0[1])
                    length = float(math.sqrt(dx * dx + dy * dy))
                    if length < 1e-6:
                        continue
                    if length > float(jump_break_m):
                        continue

                    mid = Gf.Vec3f(
                        float((p0[0] + p1[0]) * 0.5),
                        float((p0[1] + p1[1]) * 0.5),
                        float((p0[2] + p1[2]) * 0.5 + z_lift),
                    )

                    yaw = float(math.atan2(dy, dx))
                    q = _quath_from_yaw_z(yaw)
                    scale = Gf.Vec3f(float(length), float(seg_width), float(seg_height))

                    seg_positions_py.append(mid)
                    seg_orients_py.append(q)
                    seg_scales_py.append(scale)
                    proto_indices_py.append(0)

                    seg_count += 1
                    if max_segments_per_type is not None and seg_count >= int(max_segments_per_type):
                        break
                if max_segments_per_type is not None and seg_count >= int(max_segments_per_type):
                    break

            if seg_count == 0:
                continue

            instancer = UsdGeom.PointInstancer.Define(self.stage, instancer_path)
            proto_path = f"{instancer_path}/CubeProto"
            cube = UsdGeom.Cube.Define(self.stage, proto_path)
            cube.GetSizeAttr().Set(1.0)

            instancer.CreatePrototypesRel().SetTargets([cube.GetPath()])
            instancer.GetPositionsAttr().Set(Vt.Vec3fArray(seg_positions_py))
            instancer.GetOrientationsAttr().Set(Vt.QuathArray(seg_orients_py))
            instancer.GetScalesAttr().Set(Vt.Vec3fArray(seg_scales_py))
            instancer.GetProtoIndicesAttr().Set(Vt.IntArray(proto_indices_py))

            instancer.GetPrim().SetCustomDataByKey("road_type", int(t))

    # -------- spawn helpers --------
    def _spawn_vehicle_wizard_under(
        self,
        parent_path: str,
        position_m: Tuple[float, float, float],
        yaw_deg: float,
    ) -> Optional[str]:
        stage = self.stage
        _ensure_world_default_prim(stage)

        parent_xform = UsdGeom.Xform.Define(stage, parent_path)
        xform_api = UsdGeom.XformCommonAPI(parent_xform)

        unit_scale, meters_per_unit = _get_unit_scale(stage)
        x_m, y_m, z_m = position_m
        pos_units = Gf.Vec3d(x_m / meters_per_unit, y_m / meters_per_unit, z_m / meters_per_unit)

        xform_api.SetTranslate(pos_units)
        xform_api.SetRotate(Gf.Vec3f(0.0, 0.0, float(yaw_deg)), UsdGeom.XformCommonAPI.RotationOrderXYZ)

        vehicle_data = VehicleWizard.VehicleData(
            unit_scale,
            VehicleWizard.VehicleData.AXIS_Z,
            VehicleWizard.VehicleData.AXIS_X,
        )
        vehicle_data.rootVehiclePath = parent_path + "/Vehicle"
        vehicle_data.rootSharedPath = SHARED_ROOT

        ret = PhysXVehicleWizardCreateCommand.execute(vehicle_data)
        success = bool(ret[0]) if isinstance(ret, (tuple, list)) and len(ret) > 0 else False
        if not success:
            payload = ret[1] if isinstance(ret, (tuple, list)) and len(ret) > 1 else None
            messages = payload[0] if isinstance(payload, (tuple, list)) and len(payload) > 0 else None
            print("[VEH WIZ FAIL]", parent_path, "messages:", messages)
            return None

        return vehicle_data.rootVehiclePath

    def _spawn_parked_car_visual(
        self,
        path: str,
        *,
        position_m: Tuple[float, float, float],
        yaw_deg: float,
        chassis_size_m: Tuple[float, float, float] = (4.0, 2.0, 1.0),  # L,W,H
        wheel_radius_m: float = 0.35,
        wheel_thickness_m: float = 0.15,
        wheel_inset_x_m: float = 0.35,
        wheel_inset_y_m: float = 0.25,
        ground_clearance_m: float = 0.25,
    ) -> str:
        stage = self.stage
        mpu = _meters_per_unit(stage)

        L, W, H = chassis_size_m
        clearance = float(max(0.0, ground_clearance_m))

        root = UsdGeom.Xform.Define(stage, path)
        xapi = UsdGeom.XformCommonAPI(root)
        xapi.SetTranslate(Gf.Vec3d(position_m[0] / mpu, position_m[1] / mpu, position_m[2] / mpu))
        xapi.SetRotate(Gf.Vec3f(0.0, 0.0, float(yaw_deg)), UsdGeom.XformCommonAPI.RotationOrderXYZ)

        chassis = UsdGeom.Cube.Define(stage, path + "/Chassis")
        chassis.GetSizeAttr().Set(1.0)

        capi = UsdGeom.XformCommonAPI(chassis)
        capi.SetTranslate(Gf.Vec3d(0.0, 0.0, float((0.5 * H + clearance) / mpu)))
        capi.SetScale(Gf.Vec3f(float(L / mpu), float(W / mpu), float(H / mpu)))

        # cheap static collider
        UsdPhysics.CollisionAPI.Apply(chassis.GetPrim())

        wheels_root = UsdGeom.Xform.Define(stage, path + "/Wheels")

        half_L = 0.5 * L
        half_W = 0.5 * W
        wheel_inset_x_m = float(max(0.0, min(wheel_inset_x_m, half_L - 1e-3)))
        wheel_inset_y_m = float(max(0.0, min(wheel_inset_y_m, half_W - 1e-3)))

        wx_front = +(half_L - wheel_inset_x_m)
        wx_rear  = -(half_L - wheel_inset_x_m)
        wy_left  = +(half_W - wheel_inset_y_m)
        wy_right = -(half_W - wheel_inset_y_m)

        wz = float(wheel_radius_m)  # wheels touch ground
        wheel_rot = Gf.Vec3f(90.0, 0.0, 0.0)

        wheel_specs = {
            "FL": (wx_front, wy_left,  wz),
            "FR": (wx_front, wy_right, wz),
            "RL": (wx_rear,  wy_left,  wz),
            "RR": (wx_rear,  wy_right, wz),
        }

        for name, (wx, wy, wz_m) in wheel_specs.items():
            wpath = wheels_root.GetPath().pathString + f"/{name}"
            wheel = UsdGeom.Cylinder.Define(stage, wpath)
            wheel.CreateRadiusAttr(float(wheel_radius_m / mpu))
            wheel.CreateHeightAttr(float(wheel_thickness_m / mpu))

            wapi = UsdGeom.XformCommonAPI(wheel)
            wapi.SetTranslate(Gf.Vec3d(float(wx / mpu), float(wy / mpu), float(wz_m / mpu)))
            wapi.SetRotate(wheel_rot, UsdGeom.XformCommonAPI.RotationOrderXYZ)

        # tag
        root_prim = stage.GetPrimAtPath(path)
        root_prim.SetCustomDataByKey("parked_car", True)
        return path

    # -------- agents + goals --------
    def build_agents_with_goals(
        self,
        cfg: Dict[str, Any],
        max_agents: Optional[int] = None,
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,
        require_goal_in_bounds: bool = True,
        # NEW knobs:
        parked_if_start_in_goal: bool = True,
        start_goal_thresh_m: Optional[float] = None,  # if None uses goal_radius_m
        parked_ground_z_m: float = 0.0,
        parked_chassis_size_m: Tuple[float, float, float] = (4.0, 2.0, 1.0),
        parked_wheel_radius_m: float = 0.35,
        parked_wheel_thickness_m: float = 0.15,
        parked_wheel_inset_x_m: float = 0.35,
        parked_wheel_inset_y_m: float = 0.25,
        parked_ground_clearance_m: float = 0.25,
    ) -> None:
        agents = (cfg.get("agents", {}) or {}).get("items", []) or []
        kept = 0
        skipped = 0

        for idx, a in enumerate(agents):
            if max_agents is not None and kept >= int(max_agents):
                break

            s = a.get("start", {}) or {}
            e = a.get("end", {}) or {}

            sx = _safe_float(s.get("x", None))
            sy = _safe_float(s.get("y", None))
            sz = _safe_float(s.get("z", 0.0), 0.0)
            syaw = _safe_float(s.get("yaw", 0.0), 0.0)  # radians

            ex = _safe_float(e.get("x", None))
            ey = _safe_float(e.get("y", None))
            ez = _safe_float(e.get("z", 0.0), 0.0)

            if sx is None or sy is None or ex is None or ey is None:
                skipped += 1
                continue

            sx, sy, sz = self._to_local_xyz(sx, sy, sz)
            ex, ey, ez = self._to_local_xyz(ex, ey, ez)

            if not self._in_bounds_xy(float(sx), float(sy)):
                skipped += 1
                continue
            if require_goal_in_bounds and (not self._in_bounds_xy(float(ex), float(ey))):
                skipped += 1
                continue

            thresh = float(start_goal_thresh_m) if start_goal_thresh_m is not None else float(goal_radius_m)
            start_in_goal = self._is_start_within_goal(sx, sy, sz, ex, ey, ez, thresh)

            agent_id = _safe_int(a.get("agent_id", idx), idx)
            agent_path = f"{self.agents_root}/Agent_{kept:04d}_id{agent_id}"
            UsdGeom.Xform.Define(self.stage, agent_path)

            yaw_deg = float(np.degrees(float(syaw)))

            # --- NEW: parked car path ---
            if parked_if_start_in_goal and start_in_goal:
                parked_path = f"{agent_path}/ParkedCar"
                self._spawn_parked_car_visual(
                    parked_path,
                    position_m=(float(sx), float(sy), float(parked_ground_z_m)),
                    yaw_deg=yaw_deg,
                    chassis_size_m=parked_chassis_size_m,
                    wheel_radius_m=parked_wheel_radius_m,
                    wheel_thickness_m=parked_wheel_thickness_m,
                    wheel_inset_x_m=parked_wheel_inset_x_m,
                    wheel_inset_y_m=parked_wheel_inset_y_m,
                    ground_clearance_m=parked_ground_clearance_m,
                )

                prim_outer = self.stage.GetPrimAtPath(parked_path)
                prim_outer.SetCustomDataByKey("agent_id", int(agent_id))
                prim_outer.SetCustomDataByKey("agent_type", int(_safe_int(a.get("agent_type", -1), -1)))
                prim_outer.SetCustomDataByKey("track_idx", int(_safe_int(a.get("track_idx", kept), kept)))
                prim_outer.SetCustomDataByKey("start_in_goal", True)
                prim_outer.SetCustomDataByKey("controllable", False)

                # IMPORTANT: NO goal cylinder for parked cars
                kept += 1
                continue

            # --- otherwise: full PhysX vehicle + goal cylinder ---
            veh_parent = f"{agent_path}/Vehicle_Parent"
            veh_outer = self._spawn_vehicle_wizard_under(
                veh_parent,
                position_m=(float(sx), float(sy), float(spawn_z_m)),
                yaw_deg=yaw_deg,
            )
            if veh_outer is None:
                skipped += 1
                continue

            prim_outer = self.stage.GetPrimAtPath(veh_outer)
            prim_outer.SetCustomDataByKey("agent_id", int(agent_id))
            prim_outer.SetCustomDataByKey("agent_type", int(_safe_int(a.get("agent_type", -1), -1)))
            prim_outer.SetCustomDataByKey("track_idx", int(_safe_int(a.get("track_idx", kept), kept)))
            prim_outer.SetCustomDataByKey("start_in_goal", False)
            prim_outer.SetCustomDataByKey("controllable", True)

            goal_path = f"{self.goals_root}/Goal_{kept:04d}_id{agent_id}"
            cyl = UsdGeom.Cylinder.Define(self.stage, goal_path)
            cyl.CreateRadiusAttr(float(goal_radius_m))
            cyl.CreateHeightAttr(0.2)
            UsdGeom.XformCommonAPI(cyl).SetTranslate(Gf.Vec3d(float(ex), float(ey), float(ez + 0.1)))

            kept += 1

        print(f"[MiniWorldBuilder] Agents kept={kept} skipped={skipped}")

    # -------- entry point --------
    def build_from_json(
        self,
        json_path: Union[str, Path],
        *,
        max_agents: int = 50,
        # road params:
        polyline_reduction_area: float = 0.0,
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,
        # agent params:
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,
        # parked car params:
        parked_if_start_in_goal: bool = True,
        start_goal_thresh_m: Optional[float] = None,
        parked_ground_z_m: float = 0.0,
        parked_chassis_size_m: Tuple[float, float, float] = (4.0, 2.0, 1.0),
        parked_wheel_radius_m: float = 0.35,
        parked_wheel_thickness_m: float = 0.15,
        parked_wheel_inset_x_m: float = 0.35,
        parked_wheel_inset_y_m: float = 0.25,
        parked_ground_clearance_m: float = 0.25,
    ) -> None:
        p = Path(json_path).expanduser().resolve()
        with p.open("r", encoding="utf-8") as f:
            cfg = json.load(f)

        root_prim = self.stage.GetPrimAtPath(self.world_root)
        root_prim.SetCustomDataByKey("scene_json", str(p))
        root_prim.SetCustomDataByKey("origin_mode", str(self.origin_mode))
        root_prim.SetCustomDataByKey("bounds_w_m", float(self.bounds.width_m))
        root_prim.SetCustomDataByKey("bounds_l_m", float(self.bounds.length_m))

        self.build_road_segments_grouped(
            cfg,
            polyline_reduction_area=polyline_reduction_area,
            min_points_for_reduction=min_points_for_reduction,
            jump_break_m=jump_break_m,
            seg_width=seg_width,
            seg_height=seg_height,
            z_lift=z_lift,
        )
        self.build_agents_with_goals(
            cfg,
            max_agents=max_agents,
            spawn_z_m=spawn_z_m,
            goal_radius_m=goal_radius_m,
            require_goal_in_bounds=True,
            parked_if_start_in_goal=parked_if_start_in_goal,
            start_goal_thresh_m=start_goal_thresh_m,
            parked_ground_z_m=parked_ground_z_m,
            parked_chassis_size_m=parked_chassis_size_m,
            parked_wheel_radius_m=parked_wheel_radius_m,
            parked_wheel_thickness_m=parked_wheel_thickness_m,
            parked_wheel_inset_x_m=parked_wheel_inset_x_m,
            parked_wheel_inset_y_m=parked_wheel_inset_y_m,
            parked_ground_clearance_m=parked_ground_clearance_m,
        )


In [ ]:
from pathlib import Path
import omni.usd

stage = omni.usd.get_context().get_stage()

scene_dir = Path("/home/yz8733/Github/isaac-rl/data/processed/waymo_scenes_json")
json_paths = sorted(scene_dir.glob("scene_*.json"))[:16]

layout = GridLayout(world_size_m=(200.0, 200.0), padding_m=0.0, grid_cols=10, base_z_m=0.0)

ctor = ChocolateBarConstructor(
    stage=stage,
    root_container="/World/MiniWorlds",
    layout=layout,
    origin_mode="center",
)

ctor.build(
    json_paths=json_paths,
    world_count=100,
    bounds_size_m=200.0,
    max_agents_per_world=30,

    # road
    jump_break_m=3.0,
    seg_width=0.10,
    seg_height=0.10,
    z_lift=0.02,
    polyline_reduction_area=0.0,

    # agents
    spawn_z_m=1.0,
    goal_radius_m=3.0,

    # NEW: start_in_goal => parked car (NO goal cylinder)
    parked_if_start_in_goal=True,
    start_goal_thresh_m=10,          # None => uses goal_radius_m
    parked_ground_z_m=0.0,             # ground reference
    parked_chassis_size_m=(4.0, 2.0, 1.0),
    parked_wheel_radius_m=0.35,
    parked_wheel_thickness_m=0.15,
    parked_wheel_inset_x_m=0.6,
    parked_wheel_inset_y_m=0.05,
    parked_ground_clearance_m=0.25,
)


[MiniWorldBuilder] Agents kept=2 skipped=7
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=9 skipped=5
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=18 skipped=2
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=30 skipped=3
[MiniWorldBuilder] Agents kept=13 skipped=1
[MiniWorldBuilder] Agents kept=28 skipped=2
[MiniWorldBuilder] Agents kept=0 skipped=4
[MiniWorldBuilder] Agents kept=0 skipped=17
[MiniWorldBuilder] Agents kept=11 skipped=13
[MiniWorldBuilder] Agents kept=8 skipped=1
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=19 skipped=6
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=2 skipped=7
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=9 skipped=5
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=18 skipped=2
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=30 ski

In [ ]:
WAYMO_DATA_PATH = '/home/yz8733/Github/isaac-rl/data/'

In [ ]:
# chocolate_waymo_builder.py
# Build a "chocolate bar" grid of mini-worlds in Isaac Sim from Waymo-scene JSONs.
#
# ✅ Key properties:
# - Each mini-world is its own prim: /World/MiniWorlds/world_000, world_001, ...
# - The grid placement is ONLY via root-prim translation (like a chocolate bar).
# - Each mini-world enforces a local bounds box (e.g., 200m x 200m). Out-of-bounds => NOT SPAWNED.
# - Road segments are grouped by category/type (one PointInstancer per type).
# - Vehicles spawn via the SAME PhysX Vehicle Wizard API.
# - Each agent gets a destination cylinder marker (radius=3m).
#
# ✅ New behavior you asked for:
# - If an agent starts within its goal radius, we still spawn it using the wizard,
#   BUT then we convert it into a "pushable obstacle":
#     - keep collider + dynamic rigid body (so collision can move it)
#     - strip/remove the expensive vehicle/wheel/drive bits (best-effort)
#     - make it heavy + damped so it stays put unless bumped
#
# Assumed JSON schema:
# cfg["road"]["polylines"] = list of dicts with keys:
#   - "type": int
#   - "id": int
#   - "xyz": list[[x,y,z], ...]   (meters)
#
# cfg["agents"]["items"] = list of dicts with:
#   - "agent_id" (optional)
#   - "agent_type" (optional)
#   - "track_idx" (optional)
#   - "start": {"x","y","z","yaw"}   yaw in radians
#   - "end":   {"x","y","z","yaw"}   yaw optional

from __future__ import annotations

import json
import math
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional, Sequence, Tuple, Union

import numpy as np
from pxr import Gf, Sdf, Usd, UsdGeom, UsdPhysics, Vt

import omni.usd

# ---- PhysX vehicle wizard imports (THE WORKING ONES you pasted) ----
from omni.physxvehicle.scripts.wizards import physxVehicleWizard as VehicleWizard
from omni.physxvehicle.scripts.helpers.UnitScale import UnitScale
from omni.physxvehicle.scripts.commands import PhysXVehicleWizardCreateCommand

# PhysX schema (optional; we use it only for kinematic/CCD flags; safe if unavailable)
try:
    from pxr import PhysxSchema
except Exception:
    PhysxSchema = None  # type: ignore

ROOT_PATH = "/World"
SHARED_ROOT = ROOT_PATH + "/VehicleShared"


# ----------------------------
# Utilities
# ----------------------------

def _ensure_world_default_prim(stage: Usd.Stage) -> None:
    world = stage.GetPrimAtPath(ROOT_PATH)
    if not world.IsValid():
        world = UsdGeom.Xform.Define(stage, ROOT_PATH).GetPrim()
    if not stage.GetDefaultPrim().IsValid():
        stage.SetDefaultPrim(world)

def _get_unit_scale(stage: Usd.Stage) -> Tuple[UnitScale, float]:
    """Return (UnitScale, meters_per_unit)."""
    meters_per_unit = UsdGeom.GetStageMetersPerUnit(stage)
    if meters_per_unit == 0:
        meters_per_unit = 0.01  # Isaac default often cm
    length_scale = 1.0 / meters_per_unit

    kilograms_per_unit = UsdPhysics.GetStageKilogramsPerUnit(stage)
    if kilograms_per_unit == 0:
        kilograms_per_unit = 1.0
    mass_scale = 1.0 / kilograms_per_unit
    return UnitScale(length_scale, mass_scale), meters_per_unit

def _safe_int(x, default=-1) -> int:
    try:
        return int(x)
    except Exception:
        try:
            return int(float(x))
        except Exception:
            return default

def _safe_float(x, default=None) -> Optional[float]:
    try:
        return float(x)
    except Exception:
        return default

def _quath_from_yaw_z(yaw_rad: float) -> Gf.Quath:
    """Half-precision quat rotation about +Z."""
    half = 0.5 * float(yaw_rad)
    w = float(np.cos(half))
    z = float(np.sin(half))
    try:
        return Gf.Quath(Gf.Half(w), Gf.Half(0.0), Gf.Half(0.0), Gf.Half(z))
    except Exception:
        return Gf.Quath(w, 0.0, 0.0, z)

def _reduce_polyline_xy(points_xyz: np.ndarray, area_thresh: float) -> np.ndarray:
    """Iteratively remove points with small local triangle area (XY) under threshold."""
    pts = np.asarray(points_xyz, dtype=np.float32)
    if pts.shape[0] < 3 or area_thresh <= 0:
        return pts

    keep = np.ones((pts.shape[0],), dtype=bool)
    changed = True

    def tri_area_xy(a, b, c) -> float:
        abx = float(b[0] - a[0]); aby = float(b[1] - a[1])
        acx = float(c[0] - a[0]); acy = float(c[1] - a[1])
        cross = abx * acy - aby * acx
        return 0.5 * abs(cross)

    while changed:
        changed = False
        idxs = np.nonzero(keep)[0]
        if idxs.size < 3:
            break
        for k in range(1, idxs.size - 1):
            i0 = idxs[k - 1]; i1 = idxs[k]; i2 = idxs[k + 1]
            if tri_area_xy(pts[i0], pts[i1], pts[i2]) < area_thresh:
                keep[i1] = False
                changed = True
    return pts[keep]


# ----------------------------
# Mini-world builder (ONE world)
# ----------------------------

@dataclass
class LocalBounds:
    width_m: float = 200.0
    length_m: float = 200.0
    origin_xy: Tuple[float, float] = (0.0, 0.0)   # local-space center of bounds


class WaymoJsonMiniWorldBuilder:
    """
    Builds ONE mini-world under world_root (which is already placed in the big grid by root translation).

    Origin handling:
    - origin_mode="center": compute a scene center from road points, subtract it so content is near (0,0).
    - origin_mode="zero": no recentering; use JSON coords as-is.
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        world_root: str = "/World/MiniWorlds/world_000",
        bounds: Optional[LocalBounds] = None,
        origin_mode: str = "center",  # "center" or "zero"
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.world_root = world_root
        self.bounds = bounds or LocalBounds()
        self.origin_mode = origin_mode

        # internal computed origin shift (what we subtract from JSON coords)
        self._scene_center = np.zeros((3,), dtype=np.float32)

        # ensure root prim exists
        UsdGeom.Xform.Define(self.stage, self.world_root)

        # subfolders
        self.road_root = f"{self.world_root}/Road"
        self.agents_root = f"{self.world_root}/Agents"
        self.goals_root = f"{self.world_root}/Goals"
        UsdGeom.Xform.Define(self.stage, self.road_root)
        UsdGeom.Xform.Define(self.stage, self.agents_root)
        UsdGeom.Xform.Define(self.stage, self.goals_root)

    # -------- bounds gating in LOCAL coords --------
    def _in_bounds_xy(self, x: float, y: float) -> bool:
        ox, oy = self.bounds.origin_xy
        hw = 0.5 * float(self.bounds.width_m)
        hl = 0.5 * float(self.bounds.length_m)
        return (ox - hw <= x <= ox + hw) and (oy - hl <= y <= oy + hl)

    def _filter_polyline_points(self, pts: np.ndarray) -> np.ndarray:
        if pts.shape[0] == 0:
            return pts
        m = np.array([self._in_bounds_xy(float(p[0]), float(p[1])) for p in pts], dtype=bool)
        return pts[m]

    # -------- origin shift --------
    def _compute_scene_center_from_road(self, polylines: List[Dict[str, Any]]) -> np.ndarray:
        all_pts = []
        for pl in polylines:
            xyz = pl.get("xyz", None)
            if not xyz:
                continue
            arr = np.asarray(xyz, dtype=np.float32)
            if arr.ndim == 2 and arr.shape[1] >= 2 and arr.shape[0] >= 1:
                if arr.shape[1] == 2:
                    arr = np.concatenate([arr, np.zeros((arr.shape[0], 1), dtype=np.float32)], axis=1)
                all_pts.append(arr[:, :3])
        if not all_pts:
            return np.zeros((3,), dtype=np.float32)
        pts = np.concatenate(all_pts, axis=0)
        return pts.mean(axis=0)

    def _to_local_xyz(self, x: float, y: float, z: float) -> Tuple[float, float, float]:
        return (float(x - self._scene_center[0]),
                float(y - self._scene_center[1]),
                float(z - self._scene_center[2]))

    def _is_start_within_goal(self, sx: float, sy: float, sz: float, ex: float, ey: float, ez: float, radius_m: float) -> bool:
        dx = float(ex - sx)
        dy = float(ey - sy)
        dz = float(ez - sz)
        return (dx * dx + dy * dy + dz * dz) <= float(radius_m * radius_m)

    # -------- road segments grouped by type --------
    def build_road_segments_grouped(
        self,
        cfg: Dict[str, Any],
        polyline_reduction_area: float = 0.0,   # 0 disables reduction
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,
        max_segments_per_type: Optional[int] = None,
    ) -> None:
        road = (cfg.get("road", {}) or {})
        polylines = road.get("polylines", []) or []

        # compute scene center (for origin_mode="center")
        if self.origin_mode == "center":
            self._scene_center = self._compute_scene_center_from_road(polylines)
        else:
            self._scene_center = np.zeros((3,), dtype=np.float32)

        # group polylines by type
        by_type: Dict[int, List[np.ndarray]] = {}
        for pl in polylines:
            t = _safe_int(pl.get("type", -1), -1)
            xyz = pl.get("xyz", None)
            if not xyz:
                continue
            pts = np.asarray(xyz, dtype=np.float32)
            if pts.ndim != 2 or pts.shape[0] < 2:
                continue
            if pts.shape[1] == 2:
                pts = np.concatenate([pts, np.zeros((pts.shape[0], 1), dtype=np.float32)], axis=1)

            pts_local = np.array([self._to_local_xyz(p[0], p[1], p[2]) for p in pts[:, :3]], dtype=np.float32)

            pts_local = self._filter_polyline_points(pts_local)
            if pts_local.shape[0] < 2:
                continue

            if polyline_reduction_area > 0 and pts_local.shape[0] >= int(min_points_for_reduction):
                pts_local = _reduce_polyline_xy(pts_local, float(polyline_reduction_area))
                if pts_local.shape[0] < 2:
                    continue

            by_type.setdefault(t, []).append(pts_local)

        # build one PointInstancer per type
        for t, polys in sorted(by_type.items(), key=lambda kv: kv[0]):
            type_root = f"{self.road_root}/Type_{t:02d}"
            instancer_path = f"{type_root}/Segments"

            UsdGeom.Xform.Define(self.stage, type_root)

            seg_positions_py = []
            seg_orients_py = []
            seg_scales_py = []
            proto_indices_py = []

            seg_count = 0

            for poly in polys:
                for i in range(poly.shape[0] - 1):
                    p0 = poly[i]
                    p1 = poly[i + 1]

                    if not (self._in_bounds_xy(float(p0[0]), float(p0[1])) and self._in_bounds_xy(float(p1[0]), float(p1[1]))):
                        continue

                    dx = float(p1[0] - p0[0])
                    dy = float(p1[1] - p0[1])
                    length = float(math.sqrt(dx * dx + dy * dy))
                    if length < 1e-6:
                        continue

                    if length > float(jump_break_m):
                        continue

                    mid = Gf.Vec3f(
                        float((p0[0] + p1[0]) * 0.5),
                        float((p0[1] + p1[1]) * 0.5),
                        float((p0[2] + p1[2]) * 0.5 + z_lift),
                    )

                    yaw = float(math.atan2(dy, dx))
                    q = _quath_from_yaw_z(yaw)
                    scale = Gf.Vec3f(float(length), float(seg_width), float(seg_height))

                    seg_positions_py.append(mid)
                    seg_orients_py.append(q)
                    seg_scales_py.append(scale)
                    proto_indices_py.append(0)

                    seg_count += 1
                    if max_segments_per_type is not None and seg_count >= int(max_segments_per_type):
                        break
                if max_segments_per_type is not None and seg_count >= int(max_segments_per_type):
                    break

            if seg_count == 0:
                continue

            seg_positions = Vt.Vec3fArray(seg_positions_py)
            seg_orients = Vt.QuathArray(seg_orients_py)
            seg_scales = Vt.Vec3fArray(seg_scales_py)
            proto_indices = Vt.IntArray(proto_indices_py)

            instancer = UsdGeom.PointInstancer.Define(self.stage, instancer_path)
            proto_path = f"{instancer_path}/CubeProto"
            cube = UsdGeom.Cube.Define(self.stage, proto_path)
            cube.GetSizeAttr().Set(1.0)

            instancer.CreatePrototypesRel().SetTargets([cube.GetPath()])
            instancer.GetPositionsAttr().Set(seg_positions)
            instancer.GetOrientationsAttr().Set(seg_orients)
            instancer.GetScalesAttr().Set(seg_scales)
            instancer.GetProtoIndicesAttr().Set(proto_indices)

            instancer.GetPrim().SetCustomDataByKey("road_type", int(t))

    # -------- vehicles + goals --------
    def _spawn_vehicle_wizard_under(self, parent_path: str, position_m: Tuple[float, float, float], yaw_deg: float) -> Optional[str]:
        stage = self.stage
        _ensure_world_default_prim(stage)

        parent_xform = UsdGeom.Xform.Define(stage, parent_path)
        xform_api = UsdGeom.XformCommonAPI(parent_xform)

        unit_scale, meters_per_unit = _get_unit_scale(stage)

        x_m, y_m, z_m = position_m
        pos_units = Gf.Vec3d(x_m / meters_per_unit, y_m / meters_per_unit, z_m / meters_per_unit)

        xform_api.SetTranslate(pos_units)
        xform_api.SetRotate(Gf.Vec3f(0.0, 0.0, float(yaw_deg)), UsdGeom.XformCommonAPI.RotationOrderXYZ)

        vehicle_data = VehicleWizard.VehicleData(
            unit_scale,
            VehicleWizard.VehicleData.AXIS_Z,
            VehicleWizard.VehicleData.AXIS_X,
        )
        vehicle_data.rootVehiclePath = parent_path + "/Vehicle"
        vehicle_data.rootSharedPath = SHARED_ROOT

        ret = PhysXVehicleWizardCreateCommand.execute(vehicle_data)
        success = bool(ret[0]) if isinstance(ret, (tuple, list)) and len(ret) > 0 else False
        if not success:
            payload = ret[1] if isinstance(ret, (tuple, list)) and len(ret) > 1 else None
            messages = payload[0] if isinstance(payload, (tuple, list)) and len(payload) > 0 else None
            print("[VEH WIZ FAIL]", parent_path, "messages:", messages)
            return None

        return vehicle_data.rootVehiclePath

    def _convert_spawned_vehicle_to_pushable_obstacle(
        self,
        vehicle_root_path: str,
        *,
        keep_only_chassis: bool = True,
        mass_kg: float = 5000.0,
        lin_damp: float = 2.0,
        ang_damp: float = 2.0,
        enable_ccd: bool = True,
    ) -> None:
        """
        Convert a spawned PhysX vehicle into a pushable obstacle:
        - Dynamic rigid body + collider (so it can be moved by collisions)
        - Remove/strip vehicle drive/wheels/joints to reduce cost (best-effort)
        """
        stage = self.stage
        veh_prim = stage.GetPrimAtPath(vehicle_root_path)
        if not veh_prim or not veh_prim.IsValid():
            return

        # --- find chassis prim (best effort) ---
        chassis_prim = None
        for p in Usd.PrimRange(veh_prim):
            if "chassis" in p.GetName().lower():
                chassis_prim = p
                break
        if chassis_prim is None:
            for p in Usd.PrimRange(veh_prim):
                try:
                    if UsdPhysics.RigidBodyAPI(p):
                        chassis_prim = p
                        break
                except Exception:
                    pass
        if chassis_prim is None:
            chassis_prim = veh_prim

        chassis_path = chassis_prim.GetPath()

        # --- delete non-chassis branches (wheels/joints/etc) ---
        if keep_only_chassis:
            to_delete = []
            for child in veh_prim.GetChildren():
                cpath = child.GetPath()
                if chassis_path.HasPrefix(cpath):
                    continue
                to_delete.append(cpath)
            for pth in sorted(to_delete, key=lambda s: len(str(s)), reverse=True):
                try:
                    stage.RemovePrim(pth)
                except Exception:
                    pass

        # --- ensure collider + rigid body on chassis (dynamic) ---
        try:
            UsdPhysics.CollisionAPI.Apply(chassis_prim)
        except Exception:
            pass
        try:
            UsdPhysics.RigidBodyAPI.Apply(chassis_prim)
        except Exception:
            pass

        # Make sure it's NOT kinematic (so collision can move it)
        if PhysxSchema is not None:
            try:
                prb = PhysxSchema.PhysxRigidBodyAPI.Apply(chassis_prim)
                prb.CreateKinematicEnabledAttr(False)
                if enable_ccd:
                    prb.CreateEnableCCDAttr(True)
            except Exception:
                pass

        # Mass + damping to keep it mostly in place unless hit
        try:
            m = UsdPhysics.MassAPI.Apply(chassis_prim)
            m.CreateMassAttr(float(mass_kg))
        except Exception:
            pass
        try:
            rb = UsdPhysics.RigidBodyAPI(chassis_prim)
            rb.CreateLinearDampingAttr(float(lin_damp))
            rb.CreateAngularDampingAttr(float(ang_damp))
        except Exception:
            pass

        # Remove vehicle-specific applied schemas on vehicle root (best-effort)
        # (schema names can vary across versions; we only try safe metadata edits)
        for api_name in [
            "PhysxVehicleAPI",
            "PhysxVehicleControllerAPI",
            "PhysxVehicleDriveAPI",
            "PhysxVehicleTankDriveAPI",
            "PhysxVehicleAckermannSteeringAPI",
        ]:
            try:
                applied = veh_prim.GetAppliedSchemas()
                if api_name in applied:
                    md = veh_prim.GetMetadata("apiSchemas")
                    if isinstance(md, list):
                        veh_prim.SetMetadata("apiSchemas", [x for x in md if x != api_name])
            except Exception:
                pass

        # Tag so RL can ignore/control appropriately
        try:
            veh_prim.SetCustomDataByKey("pushable_obstacle", True)
            veh_prim.SetCustomDataByKey("obstacle_reason", "start_in_goal")
            veh_prim.SetCustomDataByKey("kept_chassis_path", str(chassis_path))
        except Exception:
            pass

    def build_agents_with_goals(
        self,
        cfg: Dict[str, Any],
        max_agents: Optional[int] = None,
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,
        require_goal_in_bounds: bool = True,
        # NEW knobs:
        make_pushable_if_start_in_goal: bool = True,
        start_goal_thresh_m: Optional[float] = None,  # if None uses goal_radius_m
        pushable_keep_only_chassis: bool = True,
        pushable_mass_kg: float = 5000.0,
        pushable_lin_damp: float = 2.0,
        pushable_ang_damp: float = 2.0,
        pushable_enable_ccd: bool = True,
    ) -> None:
        agents = (cfg.get("agents", {}) or {}).get("items", []) or []
        kept = 0
        skipped = 0

        for idx, a in enumerate(agents):
            if max_agents is not None and kept >= int(max_agents):
                break

            s = a.get("start", {}) or {}
            e = a.get("end", {}) or {}

            sx = _safe_float(s.get("x", None))
            sy = _safe_float(s.get("y", None))
            sz = _safe_float(s.get("z", 0.0), 0.0)
            syaw = _safe_float(s.get("yaw", 0.0), 0.0)  # radians

            ex = _safe_float(e.get("x", None))
            ey = _safe_float(e.get("y", None))
            ez = _safe_float(e.get("z", 0.0), 0.0)

            if sx is None or sy is None or ex is None or ey is None:
                skipped += 1
                continue

            # convert to LOCAL (apply origin_mode center shift)
            sx, sy, sz = self._to_local_xyz(sx, sy, sz)
            ex, ey, ez = self._to_local_xyz(ex, ey, ez)

            # bounds gate (local)
            if not self._in_bounds_xy(float(sx), float(sy)):
                skipped += 1
                continue
            if require_goal_in_bounds and (not self._in_bounds_xy(float(ex), float(ey))):
                skipped += 1
                continue

            # decide if this is "already at goal"
            thresh = float(start_goal_thresh_m) if start_goal_thresh_m is not None else float(goal_radius_m)
            start_in_goal = self._is_start_within_goal(sx, sy, sz, ex, ey, ez, thresh)

            agent_id = _safe_int(a.get("agent_id", idx), idx)
            agent_path = f"{self.agents_root}/Agent_{kept:04d}_id{agent_id}"
            UsdGeom.Xform.Define(self.stage, agent_path)

            # vehicle under agent folder
            veh_parent = f"{agent_path}/Vehicle_Parent"
            yaw_deg = float(np.degrees(float(syaw)))
            veh_outer = self._spawn_vehicle_wizard_under(
                veh_parent,
                position_m=(float(sx), float(sy), float(spawn_z_m)),
                yaw_deg=yaw_deg,
            )
            if veh_outer is None:
                skipped += 1
                continue

            # convert to pushable obstacle if starts inside goal
            if make_pushable_if_start_in_goal and start_in_goal:
                self._convert_spawned_vehicle_to_pushable_obstacle(
                    veh_outer,
                    keep_only_chassis=pushable_keep_only_chassis,
                    mass_kg=pushable_mass_kg,
                    lin_damp=pushable_lin_damp,
                    ang_damp=pushable_ang_damp,
                    enable_ccd=pushable_enable_ccd,
                )

            # tag agent metadata
            prim_outer = self.stage.GetPrimAtPath(veh_outer)
            prim_outer.SetCustomDataByKey("agent_id", int(agent_id))
            prim_outer.SetCustomDataByKey("agent_type", int(_safe_int(a.get("agent_type", -1), -1)))
            prim_outer.SetCustomDataByKey("track_idx", int(_safe_int(a.get("track_idx", kept), kept)))
            prim_outer.SetCustomDataByKey("start_in_goal", bool(start_in_goal))
            prim_outer.SetCustomDataByKey("controllable", bool(not start_in_goal))

            # destination cylinder marker
            goal_path = f"{self.goals_root}/Goal_{kept:04d}_id{agent_id}"
            cyl = UsdGeom.Cylinder.Define(self.stage, goal_path)
            cyl.CreateRadiusAttr(float(goal_radius_m))
            cyl.CreateHeightAttr(0.2)
            UsdGeom.XformCommonAPI(cyl).SetTranslate(Gf.Vec3d(float(ex), float(ey), float(ez + 0.1)))

            kept += 1

        print(f"[MiniWorldBuilder] Agents kept={kept} skipped={skipped}")

    # -------- entry point --------
    def build_from_json(
        self,
        json_path: Union[str, Path],
        *,
        max_agents: int = 50,
        # road params:
        polyline_reduction_area: float = 0.0,
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,
        # agent params:
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,
        # pushable obstacle params:
        make_pushable_if_start_in_goal: bool = True,
        start_goal_thresh_m: Optional[float] = None,
        pushable_mass_kg: float = 5000.0,
        pushable_lin_damp: float = 2.0,
        pushable_ang_damp: float = 2.0,
        pushable_enable_ccd: bool = True,
    ) -> None:
        p = Path(json_path).expanduser().resolve()
        with p.open("r", encoding="utf-8") as f:
            cfg = json.load(f)

        root_prim = self.stage.GetPrimAtPath(self.world_root)
        root_prim.SetCustomDataByKey("scene_json", str(p))
        root_prim.SetCustomDataByKey("origin_mode", str(self.origin_mode))
        root_prim.SetCustomDataByKey("bounds_w_m", float(self.bounds.width_m))
        root_prim.SetCustomDataByKey("bounds_l_m", float(self.bounds.length_m))

        self.build_road_segments_grouped(
            cfg,
            polyline_reduction_area=polyline_reduction_area,
            min_points_for_reduction=min_points_for_reduction,
            jump_break_m=jump_break_m,
            seg_width=seg_width,
            seg_height=seg_height,
            z_lift=z_lift,
        )
        self.build_agents_with_goals(
            cfg,
            max_agents=max_agents,
            spawn_z_m=spawn_z_m,
            goal_radius_m=goal_radius_m,
            require_goal_in_bounds=True,
            make_pushable_if_start_in_goal=make_pushable_if_start_in_goal,
            start_goal_thresh_m=start_goal_thresh_m,
            pushable_mass_kg=pushable_mass_kg,
            pushable_lin_damp=pushable_lin_damp,
            pushable_ang_damp=pushable_ang_damp,
            pushable_enable_ccd=pushable_enable_ccd,
        )


# ----------------------------
# Chocolate-bar multi-world constructor (GRID of mini-worlds)
# ----------------------------

@dataclass
class GridLayout:
    world_size_m: Tuple[float, float] = (200.0, 200.0)
    padding_m: float = 0.0          # 0 => chocolate tiles touching
    grid_cols: int = 5
    base_z_m: float = 0.0


class ChocolateBarConstructor:
    """
    Places N mini-world roots in a grid and spawns each from a JSON.
    IMPORTANT: The per-world filtering is LOCAL, grid translation is on the root prim only.
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        root_container: str = "/World/MiniWorlds",
        layout: GridLayout = GridLayout(),
        origin_mode: str = "center",   # recommended
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.root_container = root_container
        self.layout = layout
        self.origin_mode = origin_mode

        UsdGeom.Xform.Define(self.stage, self.root_container)

    def _world_root_path(self, i: int) -> str:
        return f"{self.root_container}/world_{i:03d}"

    def _world_translation(self, i: int) -> Tuple[float, float, float]:
        cols = int(self.layout.grid_cols)
        sx, sy = self.layout.world_size_m
        pitch_x = float(sx + self.layout.padding_m)
        pitch_y = float(sy + self.layout.padding_m)
        r = i // cols
        c = i % cols
        return (float(c * pitch_x), float(r * pitch_y), float(self.layout.base_z_m))

    def _set_root_translation(self, prim_path: str, txyz: Tuple[float, float, float]) -> None:
        prim = self.stage.GetPrimAtPath(prim_path)
        if not prim.IsValid():
            UsdGeom.Xform.Define(self.stage, prim_path)
            prim = self.stage.GetPrimAtPath(prim_path)
        xform = UsdGeom.XformCommonAPI(prim)
        xform.SetTranslate(Gf.Vec3d(*txyz))

    def clear_all(self) -> None:
        # easiest: RemovePrim the container and recreate it
        self.stage.RemovePrim(Sdf.Path(self.root_container))
        UsdGeom.Xform.Define(self.stage, self.root_container)

    def build(
        self,
        json_paths: Sequence[Union[str, Path]],
        world_count: int,
        *,
        bounds_size_m: float = 200.0,
        max_agents_per_world: int = 50,
        # road params:
        polyline_reduction_area: float = 0.0,
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,
        # agent params:
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,
        # pushable obstacle params:
        make_pushable_if_start_in_goal: bool = True,
        start_goal_thresh_m: Optional[float] = None,
        pushable_mass_kg: float = 5000.0,
        pushable_lin_damp: float = 2.0,
        pushable_ang_damp: float = 2.0,
        pushable_enable_ccd: bool = True,
    ) -> None:
        json_list = [str(Path(p).expanduser().resolve()) for p in json_paths]
        if len(json_list) == 0:
            raise ValueError("json_paths is empty")

        for i in range(int(world_count)):
            root = self._world_root_path(i)
            UsdGeom.Xform.Define(self.stage, root)

            # place root in grid (chocolate layout)
            txyz = self._world_translation(i)
            self._set_root_translation(root, txyz)

            bounds = LocalBounds(width_m=float(bounds_size_m), length_m=float(bounds_size_m), origin_xy=(0.0, 0.0))

            builder = WaymoJsonMiniWorldBuilder(
                stage=self.stage,
                world_root=root,
                bounds=bounds,
                origin_mode=self.origin_mode,
            )

            json_path = json_list[i % len(json_list)]
            builder.build_from_json(
                json_path,
                max_agents=max_agents_per_world,
                polyline_reduction_area=polyline_reduction_area,
                min_points_for_reduction=min_points_for_reduction,
                jump_break_m=jump_break_m,
                seg_width=seg_width,
                seg_height=seg_height,
                z_lift=z_lift,
                spawn_z_m=spawn_z_m,
                goal_radius_m=goal_radius_m,
                make_pushable_if_start_in_goal=make_pushable_if_start_in_goal,
                start_goal_thresh_m=start_goal_thresh_m,
                pushable_mass_kg=pushable_mass_kg,
                pushable_lin_damp=pushable_lin_damp,
                pushable_ang_damp=pushable_ang_damp,
                pushable_enable_ccd=pushable_enable_ccd,
            )

        print(f"[ChocolateBarConstructor] Built {world_count} worlds under {self.root_container}")


# ----------------------------
# Example usage (exactly your style)
# ----------------------------

from pathlib import Path
import omni.usd

# 1) Get the stage
stage = omni.usd.get_context().get_stage()

# 2) Collect jsons (or pass your own list)
scene_dir = Path("/home/yz8733/Github/isaac-rl/data/processed/waymo_scenes_json")
json_paths = sorted(scene_dir.glob("scene_*.json"))[:16]  # take first 16 for test

# 3) Build a chocolate bar
layout = GridLayout(world_size_m=(200.0, 200.0), padding_m=0.0, grid_cols=10, base_z_m=0.0)

ctor = ChocolateBarConstructor(
    stage=stage,
    root_container="/World/MiniWorlds",
    layout=layout,
    origin_mode="center",
)

ctor.build(
    json_paths=json_paths,
    world_count=100,
    bounds_size_m=200.0,
    max_agents_per_world=30,
    # road visualization tuning
    jump_break_m=3.0,
    seg_width=0.10,
    seg_height=0.10,
    z_lift=0.02,
    # polyline reduction (optional)
    polyline_reduction_area=0.0,  # try 0.1 ~ 1.0 if polylines are dense
    # ---- NEW: start-in-goal => pushable obstacle ----
    make_pushable_if_start_in_goal=True,
    start_goal_thresh_m=None,       # None => use goal_radius_m
    goal_radius_m=3.0,
    pushable_mass_kg=5000.0,        # heavier => less drift
    pushable_lin_damp=2.0,
    pushable_ang_damp=2.0,
    pushable_enable_ccd=True,
)


[MiniWorldBuilder] Agents kept=2 skipped=7
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=9 skipped=5
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=18 skipped=2
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=30 skipped=3
[MiniWorldBuilder] Agents kept=13 skipped=1
[MiniWorldBuilder] Agents kept=28 skipped=2
[MiniWorldBuilder] Agents kept=0 skipped=4
[MiniWorldBuilder] Agents kept=0 skipped=17
[MiniWorldBuilder] Agents kept=11 skipped=13
[MiniWorldBuilder] Agents kept=8 skipped=1
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=19 skipped=6
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=2 skipped=7
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=9 skipped=5
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=18 skipped=2
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=30 ski

In [ ]:
# parked_car_visual_clearance_wheels_touch.py
# Parked car = STATIC collider on chassis + visual wheels (no wheel colliders)
# Ground clearance is applied ONLY to chassis (wheels still touch ground by default).

from __future__ import annotations
from typing import Tuple, Optional

from pxr import Gf, Usd, UsdGeom, UsdPhysics
import omni.usd


def _meters_per_unit(stage: Usd.Stage) -> float:
    mpu = UsdGeom.GetStageMetersPerUnit(stage)
    return float(mpu) if mpu and float(mpu) > 0 else 0.01  # Isaac often cm


def spawn_parked_car_visual(
    stage: Optional[Usd.Stage] = None,
    path: str = "/World/ParkedCar_001",
    *,
    position_m: Tuple[float, float, float] = (0.0, 0.0, 0.0),  # ground reference point
    yaw_deg: float = 0.0,
    chassis_size_m: Tuple[float, float, float] = (4.0, 2.0, 1.0),  # L, W, H
    wheel_radius_m: float = 0.35,
    wheel_thickness_m: float = 0.15,
    wheel_inset_x_m: float = 0.35,
    wheel_inset_y_m: float = 0.25,
    ground_clearance_m: float = 0.25,  # space between chassis bottom and ground
) -> str:
    if stage is None:
        stage = omni.usd.get_context().get_stage()

    mpu = _meters_per_unit(stage)
    L, W, H = chassis_size_m
    clearance = float(max(0.0, ground_clearance_m))

    # Root at ground reference
    root = UsdGeom.Xform.Define(stage, path)
    xapi = UsdGeom.XformCommonAPI(root)
    xapi.SetTranslate(Gf.Vec3d(position_m[0] / mpu, position_m[1] / mpu, position_m[2] / mpu))
    xapi.SetRotate(Gf.Vec3f(0.0, 0.0, float(yaw_deg)), UsdGeom.XformCommonAPI.RotationOrderXYZ)

    # Chassis: lift by (H/2 + clearance) so bottom is above ground
    chassis = UsdGeom.Cube.Define(stage, path + "/Chassis")
    chassis.GetSizeAttr().Set(1.0)

    capi = UsdGeom.XformCommonAPI(chassis)
    capi.SetTranslate(Gf.Vec3d(0.0, 0.0, float((0.5 * H + clearance) / mpu)))
    capi.SetScale(Gf.Vec3f(float(L / mpu), float(W / mpu), float(H / mpu)))

    # Static collider (cheap)
    UsdPhysics.CollisionAPI.Apply(chassis.GetPrim())

    # Wheels: touching ground => wheel center z = wheel_radius (NO clearance)
    wheels_root = UsdGeom.Xform.Define(stage, path + "/Wheels")

    half_L = 0.5 * L
    half_W = 0.5 * W

    wheel_inset_x_m = float(max(0.0, min(wheel_inset_x_m, half_L - 1e-3)))
    wheel_inset_y_m = float(max(0.0, min(wheel_inset_y_m, half_W - 1e-3)))

    wx_front = +(half_L - wheel_inset_x_m)
    wx_rear  = -(half_L - wheel_inset_x_m)
    wy_left  = +(half_W - wheel_inset_y_m)
    wy_right = -(half_W - wheel_inset_y_m)

    wz = float(wheel_radius_m)  # <<< wheels touch ground

    wheel_rot = Gf.Vec3f(90.0, 0.0, 0.0)  # same as before

    wheel_specs = {
        "FL": (wx_front, wy_left,  wz),
        "FR": (wx_front, wy_right, wz),
        "RL": (wx_rear,  wy_left,  wz),
        "RR": (wx_rear,  wy_right, wz),
    }

    for name, (wx, wy, wz_m) in wheel_specs.items():
        wpath = wheels_root.GetPath().pathString + f"/{name}"
        wheel = UsdGeom.Cylinder.Define(stage, wpath)
        wheel.CreateRadiusAttr(float(wheel_radius_m / mpu))
        wheel.CreateHeightAttr(float(wheel_thickness_m / mpu))

        wapi = UsdGeom.XformCommonAPI(wheel)
        wapi.SetTranslate(Gf.Vec3d(float(wx / mpu), float(wy / mpu), float(wz_m / mpu)))
        wapi.SetRotate(wheel_rot, UsdGeom.XformCommonAPI.RotationOrderXYZ)

    return path


# ---- Run ----
stage = omni.usd.get_context().get_stage()

car_path = spawn_parked_car_visual(
    stage=stage,
    path="/World/ParkedCar_001",
    position_m=(0.0, 0.0, 0.0),
    yaw_deg=15.0,
    chassis_size_m=(4.0, 2.0, 1.0),
    wheel_radius_m=0.35,
    wheel_thickness_m=0.15,
    wheel_inset_x_m=0.6,
    wheel_inset_y_m=0.05,
    ground_clearance_m=0.25,
)

print("Spawned parked car at:", car_path)


Spawned parked car at: /World/ParkedCar_001

### Isaac_constructor class (legacy)


In [ ]:
# chocolate_waymo_builder.py
# Build a "chocolate bar" grid of mini-worlds in Isaac Sim from Waymo-scene JSONs.
#
# ✅ Key properties:
# - Each mini-world is its own prim: /World/MiniWorlds/world_000, world_001, ...
# - The grid placement is ONLY via root-prim translation (like a chocolate bar).
# - Each mini-world enforces a local bounds box (e.g., 200m x 200m). Out-of-bounds => NOT SPAWNED.
# - Road segments are grouped by category/type so the policy can "recognize category" (one instancer per type).
# - Vehicles spawn via the SAME PhysX Vehicle Wizard API you pasted (omni.physxvehicle...).
# - Each agent gets a destination cylinder marker (radius=3m).
#
# Assumed JSON schema (matches your earlier comments):
# cfg["road"]["polylines"] = list of dicts with keys:
#   - "type": int
#   - "id": int
#   - "xyz": list[[x,y,z], ...]   (meters)
#   - optional "poly_id" or similar (not required)
#
# cfg["agents"]["items"] = list of dicts with:
#   - "agent_id" (optional)
#   - "agent_type" (optional)
#   - "track_idx" (optional)
#   - "start": {"x","y","z","yaw"}   yaw in radians
#   - "end":   {"x","y","z","yaw"}   yaw in radians (yaw optional)
#
# If your JSON differs, tell me the real keys and I’ll adjust.

from __future__ import annotations

import json
import math
import os
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional, Sequence, Tuple, Union

import numpy as np
from pxr import Gf, Sdf, Usd, UsdGeom, UsdPhysics, Vt

import omni.usd

# ---- PhysX vehicle wizard imports (THE WORKING ONES you pasted) ----
from omni.physxvehicle.scripts.wizards import physxVehicleWizard as VehicleWizard
from omni.physxvehicle.scripts.helpers.UnitScale import UnitScale
from omni.physxvehicle.scripts.commands import PhysXVehicleWizardCreateCommand

ROOT_PATH = "/World"
SHARED_ROOT = ROOT_PATH + "/VehicleShared"


# ----------------------------
# Utilities
# ----------------------------

def _ensure_world_default_prim(stage: Usd.Stage) -> None:
    world = stage.GetPrimAtPath(ROOT_PATH)
    if not world.IsValid():
        world = UsdGeom.Xform.Define(stage, ROOT_PATH).GetPrim()
    if not stage.GetDefaultPrim().IsValid():
        stage.SetDefaultPrim(world)

def _get_unit_scale(stage: Usd.Stage) -> Tuple[UnitScale, float]:
    """Return (UnitScale, meters_per_unit)."""
    meters_per_unit = UsdGeom.GetStageMetersPerUnit(stage)
    if meters_per_unit == 0:
        meters_per_unit = 0.01  # Isaac default is commonly cm
    length_scale = 1.0 / meters_per_unit

    kilograms_per_unit = UsdPhysics.GetStageKilogramsPerUnit(stage)
    if kilograms_per_unit == 0:
        kilograms_per_unit = 1.0
    mass_scale = 1.0 / kilograms_per_unit
    return UnitScale(length_scale, mass_scale), meters_per_unit

def _safe_int(x, default=-1) -> int:
    try:
        return int(x)
    except Exception:
        try:
            return int(float(x))
        except Exception:
            return default

def _safe_float(x, default=None) -> Optional[float]:
    try:
        return float(x)
    except Exception:
        return default

def _quath_from_yaw_z(yaw_rad: float) -> Gf.Quath:
    """Half-precision quat rotation about +Z."""
    half = 0.5 * float(yaw_rad)
    w = float(np.cos(half))
    z = float(np.sin(half))
    try:
        return Gf.Quath(Gf.Half(w), Gf.Half(0.0), Gf.Half(0.0), Gf.Half(z))
    except Exception:
        return Gf.Quath(w, 0.0, 0.0, z)

def _reduce_polyline_xy(points_xyz: np.ndarray, area_thresh: float) -> np.ndarray:
    """Iteratively remove points with small local triangle area (XY) under threshold."""
    pts = np.asarray(points_xyz, dtype=np.float32)
    if pts.shape[0] < 3 or area_thresh <= 0:
        return pts

    keep = np.ones((pts.shape[0],), dtype=bool)
    changed = True

    def tri_area_xy(a, b, c) -> float:
        abx = float(b[0] - a[0]); aby = float(b[1] - a[1])
        acx = float(c[0] - a[0]); acy = float(c[1] - a[1])
        cross = abx * acy - aby * acx
        return 0.5 * abs(cross)

    while changed:
        changed = False
        idxs = np.nonzero(keep)[0]
        if idxs.size < 3:
            break
        for k in range(1, idxs.size - 1):
            i0 = idxs[k - 1]; i1 = idxs[k]; i2 = idxs[k + 1]
            if tri_area_xy(pts[i0], pts[i1], pts[i2]) < area_thresh:
                keep[i1] = False
                changed = True
    return pts[keep]


# ----------------------------
# Mini-world builder (ONE world)
# ----------------------------

@dataclass
class LocalBounds:
    width_m: float = 200.0
    length_m: float = 200.0
    origin_xy: Tuple[float, float] = (0.0, 0.0)   # local-space center of bounds


class WaymoJsonMiniWorldBuilder:
    """
    Builds ONE mini-world under world_root (which is already placed in the big grid by root translation).

    Origin handling (this is the part you’re unsure about):
    - origin_mode="center": compute a scene center from road points, subtract it so content is near (0,0).
    - origin_mode="zero": no recentering; use JSON coords as-is (then bounds should likely follow those coords).
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        world_root: str = "/World/MiniWorlds/world_000",
        bounds: Optional[LocalBounds] = None,
        origin_mode: str = "center",  # "center" or "zero"
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.world_root = world_root
        self.bounds = bounds or LocalBounds()
        self.origin_mode = origin_mode

        # internal computed origin shift (what we subtract from JSON coords)
        self._scene_center = np.zeros((3,), dtype=np.float32)

        # ensure root prim exists
        UsdGeom.Xform.Define(self.stage, self.world_root)

        # subfolders
        self.road_root = f"{self.world_root}/Road"
        self.agents_root = f"{self.world_root}/Agents"
        self.goals_root = f"{self.world_root}/Goals"
        UsdGeom.Xform.Define(self.stage, self.road_root)
        UsdGeom.Xform.Define(self.stage, self.agents_root)
        UsdGeom.Xform.Define(self.stage, self.goals_root)

    # -------- bounds gating in LOCAL coords --------
    def _in_bounds_xy(self, x: float, y: float) -> bool:
        ox, oy = self.bounds.origin_xy
        hw = 0.5 * float(self.bounds.width_m)
        hl = 0.5 * float(self.bounds.length_m)
        return (ox - hw <= x <= ox + hw) and (oy - hl <= y <= oy + hl)

    def _filter_polyline_points(self, pts: np.ndarray) -> np.ndarray:
        if pts.shape[0] == 0:
            return pts
        m = np.array([self._in_bounds_xy(float(p[0]), float(p[1])) for p in pts], dtype=bool)
        return pts[m]

    # -------- origin shift --------
    def _compute_scene_center_from_road(self, polylines: List[Dict[str, Any]]) -> np.ndarray:
        all_pts = []
        for pl in polylines:
            xyz = pl.get("xyz", None)
            if not xyz:
                continue
            arr = np.asarray(xyz, dtype=np.float32)
            if arr.ndim == 2 and arr.shape[1] >= 2 and arr.shape[0] >= 1:
                if arr.shape[1] == 2:
                    arr = np.concatenate([arr, np.zeros((arr.shape[0], 1), dtype=np.float32)], axis=1)
                all_pts.append(arr[:, :3])
        if not all_pts:
            return np.zeros((3,), dtype=np.float32)
        pts = np.concatenate(all_pts, axis=0)
        return pts.mean(axis=0)

    def _to_local_xyz(self, x: float, y: float, z: float) -> Tuple[float, float, float]:
        # subtract scene center if origin_mode="center"
        return (float(x - self._scene_center[0]),
                float(y - self._scene_center[1]),
                float(z - self._scene_center[2]))

    # -------- road segments grouped by type --------
    def build_road_segments_grouped(
        self,
        cfg: Dict[str, Any],
        polyline_reduction_area: float = 0.0,   # 0 disables reduction
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,
        max_segments_per_type: Optional[int] = None,
    ) -> None:
        road = (cfg.get("road", {}) or {})
        polylines = road.get("polylines", []) or []

        # compute scene center (for origin_mode="center")
        if self.origin_mode == "center":
            self._scene_center = self._compute_scene_center_from_road(polylines)
        else:
            self._scene_center = np.zeros((3,), dtype=np.float32)

        # group polylines by type
        by_type: Dict[int, List[np.ndarray]] = {}
        for pl in polylines:
            t = _safe_int(pl.get("type", -1), -1)
            xyz = pl.get("xyz", None)
            if not xyz:
                continue
            pts = np.asarray(xyz, dtype=np.float32)
            if pts.ndim != 2 or pts.shape[0] < 2:
                continue
            if pts.shape[1] == 2:
                pts = np.concatenate([pts, np.zeros((pts.shape[0], 1), dtype=np.float32)], axis=1)

            # convert to LOCAL coords
            pts_local = np.array([self._to_local_xyz(p[0], p[1], p[2]) for p in pts[:, :3]], dtype=np.float32)

            # bounds filter (local)
            pts_local = self._filter_polyline_points(pts_local)
            if pts_local.shape[0] < 2:
                continue

            # optional per-polyline reduction
            if polyline_reduction_area > 0 and pts_local.shape[0] >= int(min_points_for_reduction):
                pts_local = _reduce_polyline_xy(pts_local, float(polyline_reduction_area))
                if pts_local.shape[0] < 2:
                    continue

            by_type.setdefault(t, []).append(pts_local)

        # build one PointInstancer per type
        for t, polys in sorted(by_type.items(), key=lambda kv: kv[0]):
            type_root = f"{self.road_root}/Type_{t:02d}"
            instancer_path = f"{type_root}/Segments"

            UsdGeom.Xform.Define(self.stage, type_root)

            seg_positions_py = []
            seg_orients_py = []
            seg_scales_py = []
            proto_indices_py = []

            seg_count = 0

            for poly in polys:
                for i in range(poly.shape[0] - 1):
                    p0 = poly[i]
                    p1 = poly[i + 1]

                    # endpoint bounds gate (already point-filtered, but keep it strict)
                    if not (self._in_bounds_xy(float(p0[0]), float(p0[1])) and self._in_bounds_xy(float(p1[0]), float(p1[1]))):
                        continue

                    dx = float(p1[0] - p0[0])
                    dy = float(p1[1] - p0[1])
                    length = float(math.sqrt(dx * dx + dy * dy))
                    if length < 1e-6:
                        continue

                    # break on big jumps (prevents bad connections)
                    if length > float(jump_break_m):
                        continue

                    mid = Gf.Vec3f(
                        float((p0[0] + p1[0]) * 0.5),
                        float((p0[1] + p1[1]) * 0.5),
                        float((p0[2] + p1[2]) * 0.5 + z_lift),
                    )

                    yaw = float(math.atan2(dy, dx))
                    q = _quath_from_yaw_z(yaw)
                    scale = Gf.Vec3f(float(length), float(seg_width), float(seg_height))

                    seg_positions_py.append(mid)
                    seg_orients_py.append(q)
                    seg_scales_py.append(scale)
                    proto_indices_py.append(0)

                    seg_count += 1
                    if max_segments_per_type is not None and seg_count >= int(max_segments_per_type):
                        break
                if max_segments_per_type is not None and seg_count >= int(max_segments_per_type):
                    break

            if seg_count == 0:
                continue

            seg_positions = Vt.Vec3fArray(seg_positions_py)
            seg_orients = Vt.QuathArray(seg_orients_py)
            seg_scales = Vt.Vec3fArray(seg_scales_py)
            proto_indices = Vt.IntArray(proto_indices_py)

            instancer = UsdGeom.PointInstancer.Define(self.stage, instancer_path)
            proto_path = f"{instancer_path}/CubeProto"
            cube = UsdGeom.Cube.Define(self.stage, proto_path)
            cube.GetSizeAttr().Set(1.0)

            instancer.CreatePrototypesRel().SetTargets([cube.GetPath()])
            instancer.GetPositionsAttr().Set(seg_positions)
            instancer.GetOrientationsAttr().Set(seg_orients)
            instancer.GetScalesAttr().Set(seg_scales)
            instancer.GetProtoIndicesAttr().Set(proto_indices)

            # tag category on the instancer prim (easy to read later)
            instancer.GetPrim().SetCustomDataByKey("road_type", int(t))

    # -------- vehicles + goals --------
    def _spawn_vehicle_wizard_under(self, parent_path: str, position_m: Tuple[float, float, float], yaw_deg: float) -> Optional[str]:
        stage = self.stage
        _ensure_world_default_prim(stage)

        # create parent Xform at desired pose (LOCAL coords; root prim handles grid translation)
        parent_xform = UsdGeom.Xform.Define(stage, parent_path)
        xform_api = UsdGeom.XformCommonAPI(parent_xform)

        unit_scale, meters_per_unit = _get_unit_scale(stage)

        x_m, y_m, z_m = position_m
        pos_units = Gf.Vec3d(x_m / meters_per_unit, y_m / meters_per_unit, z_m / meters_per_unit)

        xform_api.SetTranslate(pos_units)
        xform_api.SetRotate(Gf.Vec3f(0.0, 0.0, float(yaw_deg)), UsdGeom.XformCommonAPI.RotationOrderXYZ)

        vehicle_data = VehicleWizard.VehicleData(
            unit_scale,
            VehicleWizard.VehicleData.AXIS_Z,
            VehicleWizard.VehicleData.AXIS_X,
        )
        vehicle_data.rootVehiclePath = parent_path + "/Vehicle"
        vehicle_data.rootSharedPath = SHARED_ROOT

        ret = PhysXVehicleWizardCreateCommand.execute(vehicle_data)
        success = bool(ret[0]) if isinstance(ret, (tuple, list)) and len(ret) > 0 else False
        if not success:
            payload = ret[1] if isinstance(ret, (tuple, list)) and len(ret) > 1 else None
            messages = payload[0] if isinstance(payload, (tuple, list)) and len(payload) > 0 else None
            print("[VEH WIZ FAIL]", parent_path, "messages:", messages)
            return None

        return vehicle_data.rootVehiclePath

    def _spawn_goal_cylinder(self, prim_path: str, x: float, y: float, z: float, radius_m: float = 3.0) -> None:
        cyl = UsdGeom.Cylinder.Define(self.stage, prim_path)
        cyl.CreateRadiusAttr(float(radius_m))
        cyl.CreateHeightAttr(0.2)

        xf = UsdGeom.Xform.Define(self.stage, prim_path + "_Xform")
        api = UsdGeom.XformCommonAPI(xf)
        api.SetTranslate(Gf.Vec3d(float(x), float(y), float(z + 0.1)))
        # re-parent cylinder under xform for stable transforms
        self.stage.GetPrimAtPath(prim_path).GetParent().GetChildren()
        # easiest: just set translate directly on cylinder if you prefer:
        # UsdGeom.XformCommonAPI(cyl).SetTranslate(Gf.Vec3d(x,y,z+0.1))

        # Instead of re-parenting complexity, simplest:
        api2 = UsdGeom.XformCommonAPI(cyl)
        api2.SetTranslate(Gf.Vec3d(float(x), float(y), float(z + 0.1)))

    def build_agents_with_goals(
        self,
        cfg: Dict[str, Any],
        max_agents: Optional[int] = None,
        spawn_z_m: float = 1.0,   # PhysX vehicles often need a small lift
        goal_radius_m: float = 3.0,
        require_goal_in_bounds: bool = True,
    ) -> None:
        agents = (cfg.get("agents", {}) or {}).get("items", []) or []
        kept = 0
        skipped = 0

        for idx, a in enumerate(agents):
            if max_agents is not None and kept >= int(max_agents):
                break

            s = a.get("start", {}) or {}
            e = a.get("end", {}) or {}

            sx = _safe_float(s.get("x", None))
            sy = _safe_float(s.get("y", None))
            sz = _safe_float(s.get("z", 0.0), 0.0)
            syaw = _safe_float(s.get("yaw", 0.0), 0.0)  # radians

            ex = _safe_float(e.get("x", None))
            ey = _safe_float(e.get("y", None))
            ez = _safe_float(e.get("z", 0.0), 0.0)

            if sx is None or sy is None or ex is None or ey is None:
                skipped += 1
                continue

            # convert to LOCAL (apply origin_mode center shift)
            sx, sy, sz = self._to_local_xyz(sx, sy, sz)
            ex, ey, ez = self._to_local_xyz(ex, ey, ez)

            # bounds gate (local)
            if not self._in_bounds_xy(float(sx), float(sy)):
                skipped += 1
                continue
            if require_goal_in_bounds and (not self._in_bounds_xy(float(ex), float(ey))):
                skipped += 1
                continue

            agent_id = _safe_int(a.get("agent_id", idx), idx)
            agent_path = f"{self.agents_root}/Agent_{kept:04d}_id{agent_id}"
            UsdGeom.Xform.Define(self.stage, agent_path)

            # vehicle under agent folder
            veh_parent = f"{agent_path}/Vehicle_Parent"
            yaw_deg = float(np.degrees(float(syaw)))
            veh_outer = self._spawn_vehicle_wizard_under(
                veh_parent,
                position_m=(float(sx), float(sy), float(spawn_z_m)),
                yaw_deg=yaw_deg,
            )
            if veh_outer is None:
                skipped += 1
                continue

            # tag agent metadata
            prim_outer = self.stage.GetPrimAtPath(veh_outer)
            prim_outer.SetCustomDataByKey("agent_id", int(agent_id))
            prim_outer.SetCustomDataByKey("agent_type", int(_safe_int(a.get("agent_type", -1), -1)))
            prim_outer.SetCustomDataByKey("track_idx", int(_safe_int(a.get("track_idx", kept), kept)))

            # destination cylinder (radius 3m)
            goal_path = f"{self.goals_root}/Goal_{kept:04d}_id{agent_id}"
            cyl = UsdGeom.Cylinder.Define(self.stage, goal_path)
            cyl.CreateRadiusAttr(float(goal_radius_m))
            cyl.CreateHeightAttr(0.2)
            UsdGeom.XformCommonAPI(cyl).SetTranslate(Gf.Vec3d(float(ex), float(ey), float(ez + 0.1)))

            kept += 1

        print(f"[MiniWorldBuilder] Agents kept={kept} skipped={skipped}")

    # -------- entry point --------
    def build_from_json(
        self,
        json_path: Union[str, Path],
        *,
        max_agents: int = 50,
        # road params:
        polyline_reduction_area: float = 0.0,
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,
        # agent params:
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,
    ) -> None:
        p = Path(json_path).expanduser().resolve()
        with p.open("r", encoding="utf-8") as f:
            cfg = json.load(f)

        # safe scalar metadata only
        root_prim = self.stage.GetPrimAtPath(self.world_root)
        root_prim.SetCustomDataByKey("scene_json", str(p))
        root_prim.SetCustomDataByKey("origin_mode", str(self.origin_mode))
        root_prim.SetCustomDataByKey("bounds_w_m", float(self.bounds.width_m))
        root_prim.SetCustomDataByKey("bounds_l_m", float(self.bounds.length_m))

        self.build_road_segments_grouped(
            cfg,
            polyline_reduction_area=polyline_reduction_area,
            min_points_for_reduction=min_points_for_reduction,
            jump_break_m=jump_break_m,
            seg_width=seg_width,
            seg_height=seg_height,
            z_lift=z_lift,
        )
        self.build_agents_with_goals(
            cfg,
            max_agents=max_agents,
            spawn_z_m=spawn_z_m,
            goal_radius_m=goal_radius_m,
            require_goal_in_bounds=True,
        )


# ----------------------------
# Chocolate-bar multi-world constructor (GRID of mini-worlds)
# ----------------------------

@dataclass
class GridLayout:
    world_size_m: Tuple[float, float] = (200.0, 200.0)
    padding_m: float = 0.0          # 0 => chocolate tiles touching
    grid_cols: int = 5
    base_z_m: float = 0.0


class ChocolateBarConstructor:
    """
    Places N mini-world roots in a grid and spawns each from a JSON.
    IMPORTANT: The per-world filtering is LOCAL, grid translation is on the root prim only.
    """

    def __init__(
        self,
        stage: Optional[Usd.Stage] = None,
        root_container: str = "/World/MiniWorlds",
        layout: GridLayout = GridLayout(),
        origin_mode: str = "center",   # recommended
    ):
        self.stage = stage or omni.usd.get_context().get_stage()
        self.root_container = root_container
        self.layout = layout
        self.origin_mode = origin_mode

        UsdGeom.Xform.Define(self.stage, self.root_container)

    def _world_root_path(self, i: int) -> str:
        return f"{self.root_container}/world_{i:03d}"

    def _world_translation(self, i: int) -> Tuple[float, float, float]:
        cols = int(self.layout.grid_cols)
        sx, sy = self.layout.world_size_m
        pitch_x = float(sx + self.layout.padding_m)
        pitch_y = float(sy + self.layout.padding_m)
        r = i // cols
        c = i % cols
        return (float(c * pitch_x), float(r * pitch_y), float(self.layout.base_z_m))

    def _set_root_translation(self, prim_path: str, txyz: Tuple[float, float, float]) -> None:
        prim = self.stage.GetPrimAtPath(prim_path)
        if not prim.IsValid():
            UsdGeom.Xform.Define(self.stage, prim_path)
            prim = self.stage.GetPrimAtPath(prim_path)
        xform = UsdGeom.XformCommonAPI(prim)
        xform.SetTranslate(Gf.Vec3d(*txyz))

    def clear_all(self) -> None:
        for p in self.stage.Traverse():
            # fast-ish delete approach: delete the container children by name
            pass
        # easiest: just RemovePrim the container and recreate it
        self.stage.RemovePrim(Sdf.Path(self.root_container))
        UsdGeom.Xform.Define(self.stage, self.root_container)

    def build(
        self,
        json_paths: Sequence[Union[str, Path]],
        world_count: int,
        *,
        bounds_size_m: float = 200.0,
        max_agents_per_world: int = 50,
        # road params:
        polyline_reduction_area: float = 0.0,
        min_points_for_reduction: int = 10,
        jump_break_m: float = 3.0,
        seg_width: float = 0.10,
        seg_height: float = 0.10,
        z_lift: float = 0.02,
        # agent params:
        spawn_z_m: float = 1.0,
        goal_radius_m: float = 3.0,
    ) -> None:
        json_list = [str(Path(p).expanduser().resolve()) for p in json_paths]
        if len(json_list) == 0:
            raise ValueError("json_paths is empty")

        for i in range(int(world_count)):
            root = self._world_root_path(i)
            UsdGeom.Xform.Define(self.stage, root)

            # place root in grid (chocolate layout)
            txyz = self._world_translation(i)
            self._set_root_translation(root, txyz)

            # mini-world local bounds centered at (0,0)
            bounds = LocalBounds(width_m=float(bounds_size_m), length_m=float(bounds_size_m), origin_xy=(0.0, 0.0))

            builder = WaymoJsonMiniWorldBuilder(
                stage=self.stage,
                world_root=root,
                bounds=bounds,
                origin_mode=self.origin_mode,
            )

            json_path = json_list[i % len(json_list)]
            builder.build_from_json(
                json_path,
                max_agents=max_agents_per_world,
                polyline_reduction_area=polyline_reduction_area,
                min_points_for_reduction=min_points_for_reduction,
                jump_break_m=jump_break_m,
                seg_width=seg_width,
                seg_height=seg_height,
                z_lift=z_lift,
                spawn_z_m=spawn_z_m,
                goal_radius_m=goal_radius_m,
            )

        print(f"[ChocolateBarConstructor] Built {world_count} worlds under {self.root_container}")


In [ ]:
from pathlib import Path
import omni.usd

# 1) Get the stage (this is your "my_world" equivalent)
stage = omni.usd.get_context().get_stage()

# 2) Collect jsons (or pass your own list)
scene_dir = Path("/home/yz8733/Github/isaac-rl/data/processed/waymo_scenes_json")
json_paths = sorted(scene_dir.glob("scene_*.json"))[:16]  # take first 16 for test

# 3) Build a 4x4 chocolate bar (each cell is 200m x 200m)
#from chocolate_waymo_builder import ChocolateBarConstructor, GridLayout

layout = GridLayout(world_size_m=(200.0, 200.0), padding_m=0.0, grid_cols=10, base_z_m=0.0)

ctor = ChocolateBarConstructor(
    stage=stage,
    root_container="/World/MiniWorlds",
    layout=layout,
    origin_mode="center",   # IMPORTANT default: re-center each JSON into local (0,0)
)

ctor.build(
    json_paths=json_paths,
    world_count=100,
    bounds_size_m=200.0,
    max_agents_per_world=30,
    # road visualization tuning
    jump_break_m=3.0,
    seg_width=0.10,
    seg_height=0.10,
    z_lift=0.02,
    # if you want polyline reduction (optional)
    polyline_reduction_area=0.0,  # try 0.1 ~ 1.0 if your polylines are dense
)


[MiniWorldBuilder] Agents kept=2 skipped=7
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=9 skipped=5
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=18 skipped=2
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=30 skipped=3
[MiniWorldBuilder] Agents kept=13 skipped=1
[MiniWorldBuilder] Agents kept=28 skipped=2
[MiniWorldBuilder] Agents kept=0 skipped=4
[MiniWorldBuilder] Agents kept=0 skipped=17
[MiniWorldBuilder] Agents kept=11 skipped=13
[MiniWorldBuilder] Agents kept=8 skipped=1
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=19 skipped=6
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=2 skipped=7
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=9 skipped=5
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=18 skipped=2
[MiniWorldBuilder] Agents kept=30 skipped=0
[MiniWorldBuilder] Agents kept=30 ski